In [11]:
import re
import nltk
import sys
import getopt
import codecs
import os
import struct
import timeit
import params
import collections
import math
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [12]:
param = {}
param['IGNORE_STOPWORDS'] = False
param['IGNORE_SINGLE'] = False
param['BYTE_OFFSET'] = 2
param['BYTE_SIZE'] = 2

In [22]:
class BooleanIndexer(object):
    def __init__(self,document_directory,dictionary_file,postings_file,param_dict = param):
        self.param_dict = param_dict
        self.document_directory = document_directory
        self.dictionary_file = dictionary_file
        self.postings_file = postings_file
    
    def boolean_index(self):
        docID_list = [idx for idx in range(40000)]
        dictionary = {}
        for docID in docID_list:
            file_path = self.document_directory + str(docID)
            file_path+='.txt'
            if(os.path.isfile(file_path)):
                file = codecs.open(file_path,encoding = 'utf-8')
                document = file.read()
                tokens = document.split(' ')
                #one step indexing
                for word in tokens:
                    term = word.lower()
                    term = morph.parse(term)[0].normal_form
                    if(self.param_dict['IGNORE_SINGLE'] and len(term) == 1):
                        continue
                    if term not in dictionary:
                        dictionary[term] = [docID]
                    else:
                        if(dictionary[term][-1] != docID):
                            dictionary[term].append(docID)
                file.close()
        dict_file = codecs.open(self.dictionary_file,'w',encoding='utf-8')
        post_file = open(self.postings_file,'wb')
        
        byte_offset = 0
        dict_file.write('Indexed from docIDs:')
        for docID in docID_list:
            dict_file.write(str(docID) + ',')
        dict_file.write('\n')  
        for term, postings in dictionary.items():
            block_size = len(postings)
            for docID in postings:
                post_file.write(struct.pack('H',docID))       
            dict_file.write(term + ' ' + str(block_size) + ' ' + str(byte_offset) + '\n')
            byte_offset += self.param_dict['BYTE_OFFSET'] * block_size
        dict_file.close()
        post_file.close()
        
    def position_index(self):
        docID_list = [idx for idx in range(39999)]
        dictionary = {}
        #rework
        for docID in docID_list:
            file_path = self.document_directory + str(docID)
            file_path+='.txt'
            if(os.path.isfile(file_path)):
                file = codecs.open(file_path,encoding = 'utf-8')
                document = file.read()
                tokens = document.split(' ')
                for idx in range(len(tokens)):
                    token = tokens[idx].lower()
                    dictionary.setdefault(token,{})
                    dictionary[token].setdefault(docID,[])
                    dictionary[token][docID].append(idx)   
                file.close()
        dict_file = codecs.open(self.dictionary_file,'w',encoding='utf-8')
        post_file = open(self.postings_file,'wb')
        
        byte_offset = 0
        dict_file.write('Indexed from docIDs:')
        for docID in docID_list:
            dict_file.write(str(docID) + ',')
        dict_file.write('\n')
        for term,postings in dictionary.items():
            block_size = len(postings)
            summer = 0
            for docID in sorted(postings.keys()):
                block_size_pos = len(postings[docID])
                summer += block_size_pos 
                post_file.write(struct.pack('I',docID))
                post_file.write(struct.pack('I',block_size_pos))
                for item in postings[docID]:
                    post_file.write(struct.pack('I',item))
            dict_file.write(term + ' ' + str(block_size) + ' ' + str(byte_offset) + '\n')
            byte_offset += 2 * 4 * block_size + 4 * summer
        dict_file.close()
        post_file.close()
        
    def tf_idf_pos(self):
        docID_list = [idx for idx in range(39999)]
        dictionary = {}
        dict_len = {}
        #rework
        for docID in docID_list:
            file_path = self.document_directory + str(docID)
            file_path+='.txt'
            if(os.path.isfile(file_path)):
                file = codecs.open(file_path,encoding = 'utf-8')
                document = file.read()
                tokens = document.split(' ')
                for idx in range(len(tokens)):
                    token = tokens[idx].lower()
                    token = morph.parse(token)[0].normal_form
                    dictionary.setdefault(token,{})
                    dictionary[token].setdefault(docID,[])
                    dictionary[token][docID].append(idx)
                dict_len[docID] = len(tokens)
                file.close()
        dict_file = codecs.open(self.dictionary_file,'w',encoding='utf-8')
        post_file = open(self.postings_file,'wb')
        byte_offset = 0
        dict_file.write('Indexed from docIDs:')
        for docID in docID_list:
            dict_file.write(str(docID) + ',')
        dict_file.write('\n')
        for term,postings in dictionary.items():
            block_size = len(postings)
            idf = math.log10(40000/block_size)
            summer = 0
            for docID in sorted(postings.keys()):
                block_size_pos = len(postings[docID])
                if term == 'литва':
                    print(block_size_pos)
                tf = 1 + math.log10(block_size_pos)
                summer += block_size_pos 
                post_file.write(struct.pack('I',docID))
                #post_file.write(struct.pack('I',block_size_pos))
                post_file.write(struct.pack('d',tf*idf))
                #for item in postings[docID]:
                    #post_file.write(struct.pack('I',item))
            dict_file.write(term + ' ' + str(block_size) + ' ' + str(byte_offset) + '\n')
            byte_offset +=  4 * block_size  + 8 * block_size
        dict_file.close()
        post_file.close()
        
                
        
        
            

In [155]:
start = timeit.default_timer()
search = BooleanIndexer('text\wiki','dictionary.txt','postings.txt')
search.boolean_index()
end = timeit.default_timer()
print(end-start)

53.50576205099969


In [4]:
start = timeit.default_timer()
search = BooleanIndexer('text\wiki','dictionary1.txt','postings1.txt')
search.position_index()
end = timeit.default_timer()
print(end-start)

{0: [0, 287, 874, 1169, 1347, 1358, 1376, 1491, 1535, 1643, 1729, 1771, 1871, 1955, 1990, 2208], 9: [2759], 11: [4103, 11051], 16: [9056, 16900], 19: [932, 12321, 12324], 20: [288], 22: [871, 4600], 25: [6275], 38: [1999], 49: [5694, 5696, 5698, 5700, 5702], 52: [4, 1725, 1742, 2246, 6933, 7355, 7388, 7507], 55: [1713], 68: [3786], 74: [2006, 2178], 84: [5186], 85: [1538], 125: [2172], 130: [1218, 1360], 193: [825], 214: [2721], 253: [326, 448, 1586, 1606, 2778, 2917, 2968, 6130, 6266], 325: [787], 329: [754], 333: [2427], 337: [1361], 397: [2324], 399: [1838, 10259], 413: [322], 477: [3945], 495: [1323, 1929, 2145], 556: [1031], 596: [2949], 600: [5123], 704: [620], 736: [167], 742: [61], 765: [4290, 5639], 766: [1378], 774: [3683], 777: [6916], 802: [742], 807: [560], 949: [4123], 1021: [1554], 1056: [6940], 1064: [4660], 1067: [6344], 1070: [7405], 1081: [1438], 1092: [4741, 4743, 4745], 1093: [1224], 1106: [2938], 1138: [2453], 1152: [909], 1184: [5271], 1209: [717, 1924], 1384: [4

382.802116919


In [10]:
start = timeit.default_timer()
search = BooleanIndexer('text\wiki','dictionary2.txt','postings2.txt')
search.tf_idf_pos()
end = timeit.default_timer()
print(end-start)

16
1
2
2
3
1
2
1
1
5
8
1
1
2
1
1
1
2
1
1
9
1
1
1
1
1
2
1
1
3
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
2
1
1
1
2
1
1
1
1
1
2
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
2
1
1
1
1
1
4
2
2
2
3
2
1
5
1
3
1
1
2
1
1
1
3
1
1
10
1
1
1
1
1
1
1
2
3
1
1
1
1
1
2
7
2
1
1
2
2
1
1
1
1
1
1
1
4
1
1
1
2
1
1
4
1
1
1
1
1
1
1
1
1
1
2
1
1
1
3
1
1
2
4
1
1
1
1
1
1
1
1
1
1
2
1
1
1
9
1
1
1
1
1
1
3
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
2
1
1
1
1
1
1
1
4
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
2
2
1
1
2
1
1
1
1
1
8
2
1
3
5
2
1
4
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
3
1
2
1
1
1
1
1
3
1
1
2
2
1
1
2
1
2
2
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
6
1
1
2
1
1
1
3
2
1
3
1
1
1
2
1
2
1
1
3
1
1
2
1
1
1
1
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
10
1
2
1
1
3
4
1
1
1
3
1
1
1
2
3
2
1
1
1
1
1
1
1
2
1
1
1
1
1
2
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
1
2
1
2
3
1
1
1
1
3
1
99.58486574100002


In [4]:
class Search(object):
    def __init__(self,dictionary_file,dictionary_file_tf, postings_file_tf,postings_file,queries_file,output_file,postings_with_pos = None,dictionary_file1 = None):
        self.df = dictionary_file
        self.pf = postings_file
        self.pf_pos = postings_with_pos
        self.qf = queries_file
        self.of = output_file
        self.df_pos = dictionary_file1
        self.df_tf = dictionary_file_tf
        self.pf_tf = postings_file_tf
    
    def load_dictionary(self):
        dict_stream = codecs.open(self.df,encoding = 'utf-8')
        dictionary = {}
        docIDs = []
        processed = False
        for entry in dict_stream.read().split('\n'):
            if(entry):
                if not processed:
                    #print(entry[20:-1])
                    docIDs = [int(docID) for docID in entry[20:-1].split(',')]
                    processed = True
                else:
                    token = entry.split(' ')
                    term = token[0]
                    df = int(token[1])
                    offset = int(token[2])
                    dictionary[term] = (df,offset)
        dict_stream.close()           
        return (dictionary,docIDs)
    
    def load_dictionary_pos(self):
        dict_stream = codecs.open(self.df_pos,encoding = 'utf-8')
        dictionary = {}
        docIDs = []
        processed = False
        for entry in dict_stream.read().split('\n'):
            if(entry):
                if not processed:
                    #print(entry[20:-1])
                    docIDs = [int(docID) for docID in entry[20:-1].split(',')]
                    processed = True
                else:
                    token = entry.split(' ')
                    term = token[0]
                    df = int(token[1])
                    offset = int(token[2])
                    dictionary[term] = (df,offset)
        dict_stream.close()           
        return (dictionary,docIDs)
    
    def load_dictionary_tf(self):
        dict_stream = codecs.open(self.df_tf,encoding = 'utf-8')
        dictionary = {}
        docIDs = []
        processed = False
        for entry in dict_stream.read().split('\n'):
            if(entry):
                if not processed:
                    #print(entry[20:-1])
                    docIDs = [int(docID) for docID in entry[20:-1].split(',')]
                    processed = True
                else:
                    token = entry.split(' ')
                    term = token[0]
                    df = int(token[1])
                    offset = int(token[2])
                    dictionary[term] = (df,offset)
        dict_stream.close()           
        return (dictionary,docIDs)
    
    def load_postings_tf(self,term,length,offset):
        dict_tf_idf = {}
        dict_tf_idf[term] = {}
        post_file = open(self.pf_tf,'rb')
        post_file.seek(offset)
        for idx in range(length):
            a = post_file.read(4)
            docID = struct.unpack('I',a)[0]
            print(a)
            c = post_file.read(8)
            tf_idf = struct.unpack('d',c)[0]
            print('tf_idf',tf_idf)
            dict_tf_idf[term][docID] = tf_idf
        return dict_tf_idf
    
    def ranger(self,requests,answer):
        dictionary = self.load_dictionary_tf()[0]
        tf_idf_dicts = []
        for term in requests:
            tf_idf_dicts.append(self.load_postings_tf(term,dictionary[term][0],dictionary[term][1]))
        tf = []
        for idx in range(len(answer)):
            res = 0
            for jdx in range(len(tf_idf_dicts)):
                term_ =list(tf_idf_dicts[jdx].keys())[0]
                if answer[idx] not in tf_idf_dicts[jdx][term_]:
                    print('Here')
                    continue
                else:
                    res += tf_idf_dicts[jdx][term_][answer[idx]]
            print(res)
            tf.append((answer[idx],res))
        return sorted(tf,key = lambda tup: tup[1],reverse = True)
                    
                
    
    def load_postings(self,length,offset):
        post_file = codecs.open(self.pf,'rb')
        post_file.seek(offset)
        postings = []
        for i in range(length):
            posting = post_file.read(param['BYTE_SIZE'])
            docID = struct.unpack('H',posting)[0]
            postings.append(docID)
        post_file.close()
        print(len(postings))
        return postings
    
    def load_postings_with_pos(self,length,offset):
        postings = {}
        post_file = open(self.pf_pos,'rb')
        post_file.seek(offset)
        for idx in range(length):
            a = post_file.read(4)
            docID = struct.unpack('I',a)[0]
            print(a)
            b = post_file.read(4)
            lent = struct.unpack('I',b)[0]
            print(b)
            postings[docID] = []
            for idx in range(lent):
                c = post_file.read(4)
                c = struct.unpack('I',c)[0]
                postings[docID].append(c)
        post_file.close()
        return postings
        
    #with skips
    def intersect(self,left_op,right_op):
        result = []
        idx = 0
        jdx = 0
        l_skip = int(math.sqrt(len(left_op)))
        r_skip = int(math.sqrt(len(right_op)))
        while (idx < len(left_op)) and (jdx < len(right_op)):
            if left_op[idx] == right_op[jdx]:
                result.append(left_op[idx])
                idx+=1
                jdx+=1
            elif left_op[idx] > right_op[jdx]:
                if (jdx + r_skip < len(right_op)) and (right_op[jdx + r_skip] <= left_op[idx]):
                    jdx+= r_skip
                else:
                    jdx+=1
            else:
                if (idx + l_skip < len(left_op)) and (left_op[idx + l_skip] <= right_op[jdx]):
                    idx += l_skip
                else:
                    idx+=1
        return result
    
    def position_intersect(self,left_op,right_op,intersected_keys,distance):
        answer = []
        dict_answer = {}
        print(left_op)
        for docID in intersected_keys:
            l = []
            pp1 = left_op[docID]
            pp2 = right_op[docID]
            left_plen = len(pp1)
            right_plen = len(pp2)
            idx = 0
            jdx = 0
            while idx != left_plen:
                while jdx != right_plen:
                    if abs(pp1[idx] - pp2[jdx]) <= distance:
                        l.append(pp2[jdx])
                    elif pp2[jdx] > pp1[idx]:
                        break
                    jdx += 1
                while l != [] and abs(l[0] - pp1[idx]) > distance:
                    l.remove(l[0])
                for ps in l:
                    dict_answer.setdefault(docID,[])
                    answer.append([docID,pp1[idx],ps])
                    dict_answer[docID].append(pp1[idx])
                idx +=1
        return answer,dict_answer
    
    def full_enterance_citate(self,tokens,dictionary):
        answer = self.load_postings_with_pos(dictionary[tokens[0]][0],dictionary[tokens[0]][1])
        intersected_keys = sorted(list(answer.keys()))
        postings = []
        for idx in range(1,len(tokens)):
            right_op = self.load_postings_with_pos(dictionary[tokens[idx]][0],dictionary[tokens[idx]][1])
            postings.append(right_op)
            intersected_keys = self.intersect(intersected_keys,sorted(list(right_op.keys())))
        print(intersected_keys)
        k = 1
        for idx in range(len(postings)):
            print(idx)
            _,answer = self.position_intersect(answer,postings[idx],intersected_keys,k)
            intersected_keys = answer.keys()
            k += 1
        return sorted(list(answer.keys()))
            
    
                    
    def union(self,left_op,right_op):
        result = []
        idx = 0
        jdx = 0
        while (idx < len(left_op)) or (jdx < len(right_op)):
            if (idx < len(left_op)) and (jdx < len(right_op)):
                if left_op[idx] == right_op[jdx]:
                    result.append(left_op[idx])
                    idx+=1
                    jdx+=1
                    
                elif left_op[idx] < right_op[jdx]:
                    result.append(left_op[idx])
                    idx+=1
                else:
                    result.append(right_op[jdx])
                    jdx+=1
            elif idx >= len(left_op):
                result.append(right_op[jdx])
                jdx+=1
            else:
                result.append(left_op[idx])
                idx+=1
        return result
    
    def inverse(self,op,docIDs):
        if not op:
            return docIDs
        print(op)
        result = [item for item in docIDs if item not in op]
        return result
    
    def shunting_yard(self,infix_tokens):
        precedence = {}
        precedence['NOT'] = 3
        precedence['AND'] = 2
        precedence['OR'] = 1
        precedence['('] = 0
        precedence[')'] = 0
        output = []
        operator_stack = []
        for token in infix_tokens:
            if token == '(':
                operator_stack.append(token)
            elif token == ')':
                operator = operator_stack.pop()
                while operator != '(':
                    output.append(operator)
                    operator = operator_stack.pop()
            elif token in precedence:
                if operator_stack:
                    current_operator = operator_stack[-1]
                    while operator_stack and precedence[current_operator] > precedence[token]:
                        output.append(operator_stack.pop())
                        if operator_stack:
                            current_operator = operator_stack[-1]
                            
                operator_stack.append(token)
            else:
                output.append(token.lower())
                
        while operator_stack:
            output.append(operator_stack.pop())
        return output
            
    
    def process_query(self,query,docIDs,dictionary,dictionary_pos):
        query = query.replace('(', '( ')
        query = query.replace(')',' )')
        query = query.split(' ')
        result_stack = []
        postfix_queue = collections.deque(self.shunting_yard(query))
        queue = postfix_queue.copy()
        queue = list(set(queue))
        try:
            queue.remove('AND')
        except:
            pass
        try:
            queue.remove('OR')
        except:
            pass
        try:
            queue.remove('NOT')
        except:
            pass
        try:
            queue.remove('<')
        except:
            pass
        try:
            queue.remove('>')
        except:
            pass 
                
        print(queue)
        print(postfix_queue)
        while postfix_queue:
            token = postfix_queue.popleft()
            print(token)
            result = []
            if(token != 'AND' and token != 'OR' and token != 'NOT' and token != '<'):
                if token in dictionary:
                    result = self.load_postings(dictionary[token][0],dictionary[token][1])
            elif token == 'AND':
                right_operand = result_stack.pop()
                left_operand = result_stack.pop()
                result = self.intersect(left_operand,right_operand)
                    
            elif token == 'OR':
                right_operand = result_stack.pop()
                left_operand = result_stack.pop()
                result = self.union(left_operand,right_operand)
                    
            elif token == 'NOT':
                right_operand = result_stack.pop()
                print(right_operand)
                result = self.inverse(right_operand,docIDs)
                
            elif token == '<':
                citate = []
                operand = postfix_queue.popleft()
                print(operand)
                while '>' not in operand:
                    print('Citate')
                    print(operand)
                    citate.append(operand)
                    operand = postfix_queue.popleft()
                if operand == '>':
                    c = 1
                else:
                    c = int(operand[-1])
                if c == 1:
                    result = self.full_enterance_citate(citate,dictionary_pos)
                print('res',result)
                
            result_stack.append(result)
                
        if len(result_stack) != 1:
            print(len(result_stack))
            print ("ERROR: results_stack. Please check valid query")
        answer = self.ranger(queue,result_stack[-1])
        #return result_stack.pop()
        return answer
    
    def search_bool(self,query):
        loaded_dict = self.load_dictionary()
        dictionary = loaded_dict[0]
        docIDs = loaded_dict[1]
        loaded_dict_pos = self.load_dictionary_pos()
        dictionary_pos = loaded_dict[0]
        result = self.process_query(query,docIDs,dictionary,dictionary_pos)
        for idx in range(len(result)):
            docID = str(result[idx][0])
            print('wiki' + docID + " , " + str(result[idx][1]) +  '\n')
        #эстетичный вывод
        
                    
        
                        
                
            
            
        
                

In [18]:
a = Search("dictionary.txt","postings.txt","","")

In [20]:
%time a.search_bool('или AND из AND (однако OR род)')

deque(['или', 'из', 'однако', 'род', 'OR', 'AND', 'AND'])
или
18437
из
29714
однако
12956
род
4481
OR
AND
AND
wiki0

wiki1

wiki2

wiki3

wiki5

wiki6

wiki8

wiki9

wiki10

wiki11

wiki12

wiki13

wiki14

wiki15

wiki16

wiki17

wiki18

wiki19

wiki20

wiki21

wiki22

wiki25

wiki26

wiki28

wiki29

wiki31

wiki32

wiki36

wiki38

wiki39

wiki41

wiki42

wiki43

wiki44

wiki45

wiki46

wiki47

wiki48

wiki49

wiki50

wiki51

wiki52

wiki53

wiki55

wiki56

wiki57

wiki58

wiki60

wiki61

wiki63

wiki65

wiki69

wiki70

wiki71

wiki72

wiki75

wiki76

wiki77

wiki78

wiki81

wiki83

wiki84

wiki85

wiki86

wiki87

wiki92

wiki96

wiki100

wiki101

wiki103

wiki105

wiki106

wiki107

wiki109

wiki114

wiki115

wiki117

wiki119

wiki120

wiki121

wiki123

wiki125

wiki126

wiki127

wiki129

wiki130

wiki131

wiki133

wiki144

wiki146

wiki147

wiki148

wiki150

wiki151

wiki152

wiki153

wiki154

wiki155

wiki157

wiki163

wiki165

wiki166

wiki167

wiki174

wiki175

wiki176

wiki179

wi

wiki6122

wiki6128

wiki6138

wiki6139

wiki6140

wiki6141

wiki6143

wiki6145

wiki6147

wiki6148

wiki6149

wiki6152

wiki6155

wiki6160

wiki6161

wiki6162

wiki6164

wiki6166

wiki6168

wiki6175

wiki6181

wiki6182

wiki6184

wiki6188

wiki6190

wiki6193

wiki6208

wiki6215

wiki6217

wiki6229

wiki6230

wiki6232

wiki6233

wiki6239

wiki6241

wiki6245

wiki6249

wiki6250

wiki6252

wiki6256

wiki6264

wiki6265

wiki6266

wiki6267

wiki6270

wiki6276

wiki6277

wiki6281

wiki6285

wiki6287

wiki6289

wiki6290

wiki6291

wiki6295

wiki6296

wiki6302

wiki6303

wiki6308

wiki6309

wiki6315

wiki6323

wiki6327

wiki6328

wiki6338

wiki6339

wiki6342

wiki6343

wiki6345

wiki6354

wiki6355

wiki6356

wiki6374

wiki6376

wiki6377

wiki6380

wiki6381

wiki6383

wiki6386

wiki6390

wiki6391

wiki6393

wiki6395

wiki6397

wiki6401

wiki6405

wiki6406

wiki6407

wiki6410

wiki6411

wiki6412

wiki6413

wiki6424

wiki6435

wiki6440

wiki6463

wiki6474

wiki6482

wiki6490

wiki6501

wiki6503




wiki10718

wiki10721

wiki10726

wiki10728

wiki10734

wiki10736

wiki10737

wiki10738

wiki10742

wiki10744

wiki10745

wiki10746

wiki10748

wiki10749

wiki10750

wiki10753

wiki10755

wiki10759

wiki10773

wiki10774

wiki10775

wiki10776

wiki10787

wiki10789

wiki10795

wiki10797

wiki10799

wiki10801

wiki10803

wiki10804

wiki10808

wiki10814

wiki10823

wiki10824

wiki10826

wiki10827

wiki10829

wiki10830

wiki10836

wiki10837

wiki10839

wiki10843

wiki10845

wiki10852

wiki10853

wiki10854

wiki10858

wiki10863

wiki10864

wiki10865

wiki10868

wiki10870

wiki10871

wiki10873

wiki10880

wiki10890

wiki10891

wiki10893

wiki10896

wiki10905

wiki10911

wiki10913

wiki10915

wiki10918

wiki10920

wiki10921

wiki10922

wiki10923

wiki10927

wiki10928

wiki10929

wiki10930

wiki10933

wiki10936

wiki10941

wiki10942

wiki10946

wiki10947

wiki10951

wiki10955

wiki10957

wiki10958

wiki10961

wiki10963

wiki10964

wiki10967

wiki10969

wiki10978

wiki10982

wiki10986

wiki10987

wiki16329

wiki16337

wiki16338

wiki16339

wiki16341

wiki16342

wiki16346

wiki16348

wiki16354

wiki16357

wiki16361

wiki16367

wiki16368

wiki16373

wiki16377

wiki16378

wiki16383

wiki16387

wiki16388

wiki16399

wiki16401

wiki16404

wiki16406

wiki16410

wiki16411

wiki16412

wiki16414

wiki16415

wiki16418

wiki16420

wiki16426

wiki16427

wiki16433

wiki16438

wiki16440

wiki16441

wiki16444

wiki16447

wiki16448

wiki16449

wiki16456

wiki16459

wiki16461

wiki16464

wiki16470

wiki16471

wiki16472

wiki16474

wiki16481

wiki16484

wiki16485

wiki16487

wiki16512

wiki16538

wiki16546

wiki16553

wiki16554

wiki16557

wiki16562

wiki16563

wiki16564

wiki16569

wiki16570

wiki16571

wiki16572

wiki16576

wiki16579

wiki16580

wiki16581

wiki16582

wiki16583

wiki16585

wiki16588

wiki16593

wiki16594

wiki16595

wiki16598

wiki16599

wiki16607

wiki16620

wiki16622

wiki16625

wiki16631

wiki16632

wiki16633

wiki16636

wiki16637

wiki16639

wiki16648

wiki16653

wiki16654



wiki23560

wiki23565

wiki23577

wiki23592

wiki23659

wiki23661

wiki23671

wiki23672

wiki23677

wiki23678

wiki23684

wiki23685

wiki23690

wiki23695

wiki23696

wiki23702

wiki23718

wiki23735

wiki23755

wiki23793

wiki23796

wiki23801

wiki23805

wiki23811

wiki23872

wiki23885

wiki23890

wiki23892

wiki23901

wiki23902

wiki23903

wiki23907

wiki23909

wiki23911

wiki23912

wiki23913

wiki23916

wiki23919

wiki23920

wiki23925

wiki23930

wiki23933

wiki23939

wiki23971

wiki24007

wiki24012

wiki24014

wiki24027

wiki24030

wiki24033

wiki24034

wiki24037

wiki24038

wiki24041

wiki24043

wiki24044

wiki24047

wiki24054

wiki24082

wiki24084

wiki24096

wiki24133

wiki24141

wiki24143

wiki24160

wiki24162

wiki24163

wiki24165

wiki24167

wiki24171

wiki24181

wiki24187

wiki24188

wiki24195

wiki24210

wiki24213

wiki24214

wiki24234

wiki24262

wiki24286

wiki24298

wiki24314

wiki24318

wiki24323

wiki24324

wiki24334

wiki24341

wiki24342

wiki24344

wiki24345

wiki24350

wiki32324

wiki32333

wiki32338

wiki32339

wiki32340

wiki32350

wiki32353

wiki32366

wiki32367

wiki32375

wiki32388

wiki32392

wiki32397

wiki32399

wiki32406

wiki32407

wiki32409

wiki32416

wiki32417

wiki32419

wiki32424

wiki32426

wiki32433

wiki32442

wiki32448

wiki32449

wiki32450

wiki32451

wiki32455

wiki32458

wiki32459

wiki32460

wiki32463

wiki32469

wiki32472

wiki32484

wiki32485

wiki32487

wiki32488

wiki32489

wiki32491

wiki32492

wiki32507

wiki32510

wiki32512

wiki32513

wiki32518

wiki32519

wiki32520

wiki32523

wiki32524

wiki32527

wiki32531

wiki32532

wiki32536

wiki32539

wiki32543

wiki32554

wiki32557

wiki32561

wiki32565

wiki32566

wiki32575

wiki32576

wiki32577

wiki32579

wiki32580

wiki32584

wiki32585

wiki32586

wiki32591

wiki32592

wiki32593

wiki32594

wiki32596

wiki32597

wiki32601

wiki32602

wiki32605

wiki32611

wiki32613

wiki32616

wiki32617

wiki32618

wiki32621

wiki32623

wiki32624

wiki32628

wiki32629

wiki32632

wiki32644



wiki37431

wiki37447

wiki37450

wiki37461

wiki37462

wiki37486

wiki37491

wiki37501

wiki37530

wiki37531

wiki37553

wiki37576

wiki37585

wiki37601

wiki37627

wiki37658

wiki37699

wiki37700

wiki37735

wiki37736

wiki37755

wiki37788

wiki37793

wiki37813

wiki37814

wiki37821

wiki37838

wiki37854

wiki37867

wiki37887

wiki37915

wiki37917

wiki38002

wiki38010

wiki38021

wiki38074

wiki38082

wiki38097

wiki38112

wiki38159

wiki38164

wiki38173

wiki38181

wiki38210

wiki38237

wiki38244

wiki38256

wiki38261

wiki38263

wiki38264

wiki38268

wiki38270

wiki38284

wiki38292

wiki38310

wiki38311

wiki38312

wiki38314

wiki38322

wiki38326

wiki38368

wiki38399

wiki38418

wiki38475

wiki38485

wiki38497

wiki38520

wiki38529

wiki38547

wiki38560

wiki38566

wiki38567

wiki38611

wiki38641

wiki38657

wiki38753

wiki38756

wiki38771

wiki38831

wiki38842

wiki38856

wiki38867

wiki38878

wiki38952

wiki38972

wiki38992

wiki39004

wiki39049

wiki39084

wiki39085

wiki39117

In [175]:
%time a.search_bool('(всю AND (жизнь AND служить)) AND государству')

deque(['всю', 'жизнь', 'служить', 'AND', 'AND', 'государству', 'AND'])
3252
5701
1194
511
wiki11

wiki25

wiki643

wiki803

wiki1347

wiki1388

wiki1400

wiki1438

wiki1443

wiki1896

wiki3908

wiki5814

wiki6055

wiki6903

wiki7920

wiki8445

wiki9367

wiki9971

wiki10326

wiki11180

wiki11248

wiki12673

wiki14459

wiki14793

wiki17725

wiki18737

wiki19180

wiki19187

wiki21932

wiki26554

wiki27953

wiki28113

wiki28717

wiki29057

wiki33535

wiki34733

Wall time: 1.82 s


In [62]:
b = Search("dictionary.txt","postings.txt","","","postings1.txt","dictionary1.txt")

In [65]:
%time b.search_bool('< литва > OR официальное')

deque(['<', 'литва', '>', 'официальное', 'OR'])
<
литва
Citate
литва
b'\x00\x00\x00\x00'
b'\x10\x00\x00\x00'
b'\t\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\x0b\x00\x00\x00'
b'\x02\x00\x00\x00'
b'\x10\x00\x00\x00'
b'\x02\x00\x00\x00'
b'\x13\x00\x00\x00'
b'\x03\x00\x00\x00'
b'\x14\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\x16\x00\x00\x00'
b'\x02\x00\x00\x00'
b'\x19\x00\x00\x00'
b'\x01\x00\x00\x00'
b'&\x00\x00\x00'
b'\x01\x00\x00\x00'
b'1\x00\x00\x00'
b'\x05\x00\x00\x00'
b'4\x00\x00\x00'
b'\x08\x00\x00\x00'
b'7\x00\x00\x00'
b'\x01\x00\x00\x00'
b'D\x00\x00\x00'
b'\x01\x00\x00\x00'
b'J\x00\x00\x00'
b'\x02\x00\x00\x00'
b'T\x00\x00\x00'
b'\x01\x00\x00\x00'
b'U\x00\x00\x00'
b'\x01\x00\x00\x00'
b'}\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\x82\x00\x00\x00'
b'\x02\x00\x00\x00'
b'\xc1\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\xd6\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\xfd\x00\x00\x00'
b'\t\x00\x00\x00'
b'E\x01\x00\x00'
b'\x01\x00\x00\x00'
b'I\x01\x00\x00'
b'\x01\x00\x00\x00'
b'M\x01\x00\x00'
b'\x01\x00\x00\x00'
b'Q\x01\x00

wiki27063

wiki27103

wiki27117

wiki27120

wiki27193

wiki27204

wiki27277

wiki27289

wiki27345

wiki27351

wiki27354

wiki27374

wiki27450

wiki27459

wiki27465

wiki27563

wiki27629

wiki27631

wiki27632

wiki27730

wiki27768

wiki27792

wiki27883

wiki28023

wiki28113

wiki28304

wiki28310

wiki28365

wiki28376

wiki28408

wiki28464

wiki28488

wiki28510

wiki28512

wiki28530

wiki28574

wiki28579

wiki28588

wiki28591

wiki28619

wiki28620

wiki28648

wiki28680

wiki28713

wiki28762

wiki28772

wiki28799

wiki28808

wiki28828

wiki28852

wiki28869

wiki28876

wiki28882

wiki28891

wiki28907

wiki28912

wiki28926

wiki28929

wiki28977

wiki28988

wiki29061

wiki29071

wiki29124

wiki29183

wiki29203

wiki29217

wiki29263

wiki29326

wiki29497

wiki29787

wiki29795

wiki29970

wiki30015

wiki30018

wiki30042

wiki30047

wiki30064

wiki30067

wiki30068

wiki30098

wiki30172

wiki30178

wiki30205

wiki30267

wiki30287

wiki30291

wiki30383

wiki30388

wiki30406

wiki30538

wiki30540


In [66]:
%time b.search_bool('(всю AND (жизнь AND служить)) AND государству')

deque(['всю', 'жизнь', 'служить', 'AND', 'AND', 'государству', 'AND'])
всю
3252
жизнь
5701
служить
1194
AND
AND
государству
511
AND
wiki11

wiki25

wiki643

wiki803

wiki1347

wiki1388

wiki1400

wiki1438

wiki1443

wiki1896

wiki3908

wiki5814

wiki6055

wiki6903

wiki7920

wiki8445

wiki9367

wiki9971

wiki10326

wiki11180

wiki11248

wiki12673

wiki14459

wiki14793

wiki17725

wiki18737

wiki19180

wiki19187

wiki21932

wiki26554

wiki27953

wiki28113

wiki28717

wiki29057

wiki33535

wiki34733

Wall time: 3.72 s


In [5]:
a = Search("dictionary.txt","dictionary2.txt","postings2.txt","postings.txt","","","postings1.txt","dictionary1.txt")

In [13]:
%time a.search_bool('< литва > AND государству')

['литва', 'государству']
deque(['<', 'литва', '>', 'государству', 'AND'])
<
литва
Citate
литва
b'\x00\x00\x00\x00'
b'\x10\x00\x00\x00'
b'\t\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\x0b\x00\x00\x00'
b'\x02\x00\x00\x00'
b'\x10\x00\x00\x00'
b'\x02\x00\x00\x00'
b'\x13\x00\x00\x00'
b'\x03\x00\x00\x00'
b'\x14\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\x16\x00\x00\x00'
b'\x02\x00\x00\x00'
b'\x19\x00\x00\x00'
b'\x01\x00\x00\x00'
b'&\x00\x00\x00'
b'\x01\x00\x00\x00'
b'1\x00\x00\x00'
b'\x05\x00\x00\x00'
b'4\x00\x00\x00'
b'\x08\x00\x00\x00'
b'7\x00\x00\x00'
b'\x01\x00\x00\x00'
b'D\x00\x00\x00'
b'\x01\x00\x00\x00'
b'J\x00\x00\x00'
b'\x02\x00\x00\x00'
b'T\x00\x00\x00'
b'\x01\x00\x00\x00'
b'U\x00\x00\x00'
b'\x01\x00\x00\x00'
b'}\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\x82\x00\x00\x00'
b'\x02\x00\x00\x00'
b'\xc1\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\xd6\x00\x00\x00'
b'\x01\x00\x00\x00'
b'\xfd\x00\x00\x00'
b'\t\x00\x00\x00'
b'E\x01\x00\x00'
b'\x01\x00\x00\x00'
b'I\x01\x00\x00'
b'\x01\x00\x00\x00'
b'M\x01\x00\x00'
b'\x0

b'\x00\x00\x00\x00'
tf_idf 4.29762322913583
b'\t\x00\x00\x00'
tf_idf 1.9498136503246393
b'\x0b\x00\x00\x00'
tf_idf 2.536766045027437
b'\x10\x00\x00\x00'
tf_idf 2.536766045027437
b'\x13\x00\x00\x00'
tf_idf 2.8801111856370563
b'\x14\x00\x00\x00'
tf_idf 1.9498136503246393
b'\x16\x00\x00\x00'
tf_idf 2.536766045027437
b'\x19\x00\x00\x00'
tf_idf 1.9498136503246393
b'&\x00\x00\x00'
tf_idf 1.9498136503246393
b'1\x00\x00\x00'
tf_idf 3.312674905946481
b'4\x00\x00\x00'
tf_idf 3.7106708344330315
b'7\x00\x00\x00'
tf_idf 1.9498136503246393
b'D\x00\x00\x00'
tf_idf 1.9498136503246393
b'J\x00\x00\x00'
tf_idf 2.536766045027437
b'T\x00\x00\x00'
tf_idf 1.9498136503246393
b'U\x00\x00\x00'
tf_idf 1.9498136503246393
b'}\x00\x00\x00'
tf_idf 1.9498136503246393
b'\x82\x00\x00\x00'
tf_idf 2.536766045027437
b'\xc1\x00\x00\x00'
tf_idf 1.9498136503246393
b'\xd6\x00\x00\x00'
tf_idf 1.9498136503246393
b'\xfd\x00\x00\x00'
tf_idf 3.8104087209494732
b'E\x01\x00\x00'
tf_idf 1.9498136503246393
b'I\x01\x00\x00'
tf_idf 1.94

tf_idf 1.8936390911932497
b'C\x05\x00\x00'
tf_idf 2.463681258604299
b'[\x05\x00\x00'
tf_idf 1.8936390911932497
b'`\x05\x00\x00'
tf_idf 1.8936390911932497
b'l\x05\x00\x00'
tf_idf 1.8936390911932497
b'n\x05\x00\x00'
tf_idf 2.463681258604299
b'x\x05\x00\x00'
tf_idf 1.8936390911932497
b'\x9c\x05\x00\x00'
tf_idf 1.8936390911932497
b'\x9e\x05\x00\x00'
tf_idf 1.8936390911932497
b'\xa3\x05\x00\x00'
tf_idf 2.463681258604299
b'\xab\x05\x00\x00'
tf_idf 3.0337234260153485
b'\xae\x05\x00\x00'
tf_idf 1.8936390911932497
b'\xb8\x05\x00\x00'
tf_idf 1.8936390911932497
b'=\x06\x00\x00'
tf_idf 2.463681258604299
b'\x85\x06\x00\x00'
tf_idf 1.8936390911932497
b'\xbe\x06\x00\x00'
tf_idf 1.8936390911932497
b'\xc6\x06\x00\x00'
tf_idf 1.8936390911932497
b';\x07\x00\x00'
tf_idf 1.8936390911932497
b'\\\x07\x00\x00'
tf_idf 1.8936390911932497
b'^\x07\x00\x00'
tf_idf 1.8936390911932497
b'h\x07\x00\x00'
tf_idf 1.8936390911932497
b'y\x07\x00\x00'
tf_idf 1.8936390911932497
b'\x92\x07\x00\x00'
tf_idf 1.8936390911932497
b

Wall time: 5.46 s


In [14]:
a.search_bool('сибирская OR платформа')

['платформа', 'сибирская']
deque(['сибирская', 'платформа', 'OR'])
сибирская
220
платформа
398
OR
b'\x01\x00\x00\x00'
tf_idf 2.957458083139622
b'\x18\x00\x00\x00'
tf_idf 2.0021769192542744
b"'\x00\x00\x00"
tf_idf 2.957458083139622
b')\x00\x00\x00'
tf_idf 3.4016385291869113
b'1\x00\x00\x00'
tf_idf 2.0021769192542744
b'6\x00\x00\x00'
tf_idf 2.0021769192542744
b'>\x00\x00\x00'
tf_idf 2.0021769192542744
b'D\x00\x00\x00'
tf_idf 2.0021769192542744
b'M\x00\x00\x00'
tf_idf 2.0021769192542744
b'd\x00\x00\x00'
tf_idf 2.0021769192542744
b'\x90\x00\x00\x00'
tf_idf 2.0021769192542744
b' \x01\x00\x00'
tf_idf 2.0021769192542744
b'\xdd\x01\x00\x00'
tf_idf 2.604892228575912
b'\x04\x02\x00\x00'
tf_idf 2.0021769192542744
b'\x05\x02\x00\x00'
tf_idf 2.0021769192542744
b'b\x02\x00\x00'
tf_idf 2.0021769192542744
b'\x05\x03\x00\x00'
tf_idf 2.0021769192542744
b'\t\x03\x00\x00'
tf_idf 2.0021769192542744
b'L\x03\x00\x00'
tf_idf 2.0021769192542744
b'\xb4\x03\x00\x00'
tf_idf 2.0021769192542744
b'\xf0\x03\x00\x00'


Here
2.2596373105057563
Here
2.2596373105057563
Here
2.9398559202894745
Here
2.2596373105057563
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.2596373105057563
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.9398559202894745
Here
2.957458083139622
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.2596373105057563
Here
2.2596373105057563
Here
3.337758299305626
Here
2.2596373105057563
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.604892228575912
Here
3.337758299305626
Here
2.604892228575912
Here
2.604892228575912
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.2596373105057563
Here
2.2596373105057563
Here
2.9398559202894745
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.604892228575912
Here
2.957458083139622
Here
2.0021769192542744
Here
2.2596373105057563
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.0021769192542744
Here
2.0021769192542744


wiki4580 , 2.2596373105057563

wiki4637 , 2.2596373105057563

wiki4686 , 2.2596373105057563

wiki4773 , 2.2596373105057563

wiki4844 , 2.2596373105057563

wiki5071 , 2.2596373105057563

wiki5074 , 2.2596373105057563

wiki5159 , 2.2596373105057563

wiki5584 , 2.2596373105057563

wiki5650 , 2.2596373105057563

wiki6138 , 2.2596373105057563

wiki6579 , 2.2596373105057563

wiki6587 , 2.2596373105057563

wiki6610 , 2.2596373105057563

wiki6611 , 2.2596373105057563

wiki6967 , 2.2596373105057563

wiki7035 , 2.2596373105057563

wiki7302 , 2.2596373105057563

wiki7330 , 2.2596373105057563

wiki7375 , 2.2596373105057563

wiki7396 , 2.2596373105057563

wiki7406 , 2.2596373105057563

wiki7465 , 2.2596373105057563

wiki7799 , 2.2596373105057563

wiki8057 , 2.2596373105057563

wiki8061 , 2.2596373105057563

wiki8117 , 2.2596373105057563

wiki8401 , 2.2596373105057563

wiki8609 , 2.2596373105057563

wiki8917 , 2.2596373105057563

wiki9300 , 2.2596373105057563

wiki9328 , 2.2596373105057563

wiki9486

In [15]:
a.search_bool('сибирская AND платформа')

['платформа', 'сибирская']
deque(['сибирская', 'платформа', 'AND'])
сибирская
220
платформа
398
AND
b'\x01\x00\x00\x00'
tf_idf 2.957458083139622
b'\x18\x00\x00\x00'
tf_idf 2.0021769192542744
b"'\x00\x00\x00"
tf_idf 2.957458083139622
b')\x00\x00\x00'
tf_idf 3.4016385291869113
b'1\x00\x00\x00'
tf_idf 2.0021769192542744
b'6\x00\x00\x00'
tf_idf 2.0021769192542744
b'>\x00\x00\x00'
tf_idf 2.0021769192542744
b'D\x00\x00\x00'
tf_idf 2.0021769192542744
b'M\x00\x00\x00'
tf_idf 2.0021769192542744
b'd\x00\x00\x00'
tf_idf 2.0021769192542744
b'\x90\x00\x00\x00'
tf_idf 2.0021769192542744
b' \x01\x00\x00'
tf_idf 2.0021769192542744
b'\xdd\x01\x00\x00'
tf_idf 2.604892228575912
b'\x04\x02\x00\x00'
tf_idf 2.0021769192542744
b'\x05\x02\x00\x00'
tf_idf 2.0021769192542744
b'b\x02\x00\x00'
tf_idf 2.0021769192542744
b'\x05\x03\x00\x00'
tf_idf 2.0021769192542744
b'\t\x03\x00\x00'
tf_idf 2.0021769192542744
b'L\x03\x00\x00'
tf_idf 2.0021769192542744
b'\xb4\x03\x00\x00'
tf_idf 2.0021769192542744
b'\xf0\x03\x00\x00

wiki7957 , 6.9754349922289665

wiki2056 , 6.739396828492538

wiki1 , 5.897314003429097

wiki14378 , 5.661275839692667

wiki32007 , 4.942032839543749



In [8]:
a.search_bool('лесной OR кот')

['кот', 'лесной']
deque(['лесной', 'кот', 'OR'])
лесной
550
кот
337
OR
b'\x01\x00\x00\x00'
tf_idf 2.6988957715920137
b'=\x00\x00\x00'
tf_idf 2.074430090456624
b'D\x00\x00\x00'
tf_idf 2.074430090456624
b'F\x00\x00\x00'
tf_idf 2.074430090456624
b'I\x01\x00\x00'
tf_idf 2.074430090456624
b'Q\x01\x00\x00'
tf_idf 2.074430090456624
b'\x90\x01\x00\x00'
tf_idf 2.074430090456624
b'\xba\x01\x00\x00'
tf_idf 3.064184778043511
b'1\x02\x00\x00'
tf_idf 2.074430090456624
b'\t\x03\x00\x00'
tf_idf 2.6988957715920137
b'\x10\x03\x00\x00'
tf_idf 2.074430090456624
b'd\x03\x00\x00'
tf_idf 2.074430090456624
b'f\x03\x00\x00'
tf_idf 2.074430090456624
b'\xc3\x03\x00\x00'
tf_idf 2.6988957715920137
b'\xde\x03\x00\x00'
tf_idf 2.074430090456624
b'\xe1\x03\x00\x00'
tf_idf 2.074430090456624
b'\xe6\x03\x00\x00'
tf_idf 2.074430090456624
b'\xf0\x03\x00\x00'
tf_idf 2.074430090456624
b' \x04\x00\x00'
tf_idf 2.6988957715920137
b'#\x04\x00\x00'
tf_idf 2.074430090456624
b'(\x04\x00\x00'
tf_idf 2.074430090456624
b'C\x04\x00\x00

tf_idf 1.8616973018337186
b'\xc5G\x00\x00'
tf_idf 2.4221240325323685
b'\xcaG\x00\x00'
tf_idf 1.8616973018337186
b'\xdaG\x00\x00'
tf_idf 1.8616973018337186
b'\xb2H\x00\x00'
tf_idf 2.4221240325323685
b'\xbdH\x00\x00'
tf_idf 1.8616973018337186
b'\xbeH\x00\x00'
tf_idf 1.8616973018337186
b'\xcfH\x00\x00'
tf_idf 1.8616973018337186
b'\xe7H\x00\x00'
tf_idf 2.7499526543928323
b'\xebH\x00\x00'
tf_idf 3.162967872968787
b'\xf1H\x00\x00'
tf_idf 1.8616973018337186
b'\xf9H\x00\x00'
tf_idf 1.8616973018337186
b'\x16I\x00\x00'
tf_idf 1.8616973018337186
b'\x1bI\x00\x00'
tf_idf 1.8616973018337186
b'\x1dI\x00\x00'
tf_idf 1.8616973018337186
b')I\x00\x00'
tf_idf 1.8616973018337186
b'1I\x00\x00'
tf_idf 1.8616973018337186
b'\x81I\x00\x00'
tf_idf 2.4221240325323685
b'\xfdI\x00\x00'
tf_idf 3.162967872968787
b'\x04J\x00\x00'
tf_idf 1.8616973018337186
b'\x0eJ\x00\x00'
tf_idf 2.9825507632310186
b'\x1eJ\x00\x00'
tf_idf 1.8616973018337186
b'@J\x00\x00'
tf_idf 1.8616973018337186
b'VJ\x00\x00'
tf_idf 1.8616973018337186

Here
3.3103793850914824
Here
1.8616973018337186
Here
2.4221240325323685
Here
1.8616973018337186
Here
1.8616973018337186
Here
1.8616973018337186
Here
1.8616973018337186
Here
2.074430090456624
Here
2.4221240325323685
Here
2.074430090456624
Here
1.8616973018337186
Here
1.8616973018337186
Here
2.074430090456624
Here
2.074430090456624
Here
2.074430090456624
Here
1.8616973018337186
Here
2.074430090456624
Here
2.4221240325323685
Here
1.8616973018337186
Here
2.4221240325323685
Here
1.8616973018337186
Here
2.074430090456624
Here
2.074430090456624
Here
1.8616973018337186
Here
1.8616973018337186
Here
1.8616973018337186
Here
2.074430090456624
Here
1.8616973018337186
Here
1.8616973018337186
Here
1.8616973018337186
Here
1.8616973018337186
Here
3.064184778043511
Here
2.074430090456624
Here
2.074430090456624
Here
1.8616973018337186
Here
1.8616973018337186
Here
2.074430090456624
4.496554122988993
Here
1.8616973018337186
Here
1.8616973018337186
Here
1.8616973018337186
Here
1.8616973018337186
Here
2.0744

wiki19031 , 2.6988957715920137

wiki19032 , 2.6988957715920137

wiki21360 , 2.6988957715920137

wiki21387 , 2.6988957715920137

wiki21416 , 2.6988957715920137

wiki21436 , 2.6988957715920137

wiki21606 , 2.6988957715920137

wiki24427 , 2.6988957715920137

wiki24758 , 2.6988957715920137

wiki25132 , 2.6988957715920137

wiki25183 , 2.6988957715920137

wiki25513 , 2.6988957715920137

wiki25809 , 2.6988957715920137

wiki26334 , 2.6988957715920137

wiki27775 , 2.6988957715920137

wiki28020 , 2.6988957715920137

wiki28256 , 2.6988957715920137

wiki29336 , 2.6988957715920137

wiki31153 , 2.6988957715920137

wiki31753 , 2.6988957715920137

wiki32155 , 2.6988957715920137

wiki32487 , 2.6988957715920137

wiki32789 , 2.6988957715920137

wiki32995 , 2.6988957715920137

wiki33144 , 2.6988957715920137

wiki33481 , 2.6988957715920137

wiki36201 , 2.6988957715920137

wiki36839 , 2.6988957715920137

wiki37052 , 2.6988957715920137

wiki37793 , 2.6988957715920137

wiki39845 , 2.6988957715920137

wiki282 

In [9]:
a.search_bool('лесной AND кот')

['кот', 'лесной']
deque(['лесной', 'кот', 'AND'])
лесной
550
кот
337
AND
b'\x01\x00\x00\x00'
tf_idf 2.6988957715920137
b'=\x00\x00\x00'
tf_idf 2.074430090456624
b'D\x00\x00\x00'
tf_idf 2.074430090456624
b'F\x00\x00\x00'
tf_idf 2.074430090456624
b'I\x01\x00\x00'
tf_idf 2.074430090456624
b'Q\x01\x00\x00'
tf_idf 2.074430090456624
b'\x90\x01\x00\x00'
tf_idf 2.074430090456624
b'\xba\x01\x00\x00'
tf_idf 3.064184778043511
b'1\x02\x00\x00'
tf_idf 2.074430090456624
b'\t\x03\x00\x00'
tf_idf 2.6988957715920137
b'\x10\x03\x00\x00'
tf_idf 2.074430090456624
b'd\x03\x00\x00'
tf_idf 2.074430090456624
b'f\x03\x00\x00'
tf_idf 2.074430090456624
b'\xc3\x03\x00\x00'
tf_idf 2.6988957715920137
b'\xde\x03\x00\x00'
tf_idf 2.074430090456624
b'\xe1\x03\x00\x00'
tf_idf 2.074430090456624
b'\xe6\x03\x00\x00'
tf_idf 2.074430090456624
b'\xf0\x03\x00\x00'
tf_idf 2.074430090456624
b' \x04\x00\x00'
tf_idf 2.6988957715920137
b'#\x04\x00\x00'
tf_idf 2.074430090456624
b'(\x04\x00\x00'
tf_idf 2.074430090456624
b'C\x04\x00\x

tf_idf 1.8616973018337186
b'o1\x00\x00'
tf_idf 2.4221240325323685
b'\xc61\x00\x00'
tf_idf 2.4221240325323685
b'\x0c2\x00\x00'
tf_idf 1.8616973018337186
b'W2\x00\x00'
tf_idf 2.4221240325323685
b'b2\x00\x00'
tf_idf 2.4221240325323685
b'j2\x00\x00'
tf_idf 1.8616973018337186
b'\xd02\x00\x00'
tf_idf 1.8616973018337186
b'\xf92\x00\x00'
tf_idf 1.8616973018337186
b'\x0e3\x00\x00'
tf_idf 1.8616973018337186
b'!3\x00\x00'
tf_idf 1.8616973018337186
b'^3\x00\x00'
tf_idf 2.7499526543928323
b'\x8a3\x00\x00'
tf_idf 2.4221240325323685
b'\x973\x00\x00'
tf_idf 1.8616973018337186
b'\x9c3\x00\x00'
tf_idf 1.8616973018337186
b'\x074\x00\x00'
tf_idf 1.8616973018337186
b'\t4\x00\x00'
tf_idf 1.8616973018337186
b'54\x00\x00'
tf_idf 1.8616973018337186
b'r4\x00\x00'
tf_idf 2.7499526543928323
b'\x8f4\x00\x00'
tf_idf 1.8616973018337186
b'\xad4\x00\x00'
tf_idf 1.8616973018337186
b'\x835\x00\x00'
tf_idf 1.8616973018337186
b'-7\x00\x00'
tf_idf 1.8616973018337186
b'T7\x00\x00'
tf_idf 2.4221240325323685
b'b7\x00\x00'
tf_

wiki14562 , 4.496554122988993

wiki16646 , 4.496554122988993

wiki61 , 3.9361273922903424

wiki1008 , 3.9361273922903424

wiki1260 , 3.9361273922903424

wiki1261 , 3.9361273922903424

wiki4735 , 3.9361273922903424

wiki19752 , 3.9361273922903424

wiki21870 , 3.9361273922903424

wiki31217 , 3.9361273922903424

wiki33686 , 3.9361273922903424



In [14]:
a.search_bool('советского OR государства')

['советского', 'государства']
deque(['советского', 'государства', 'OR'])
советского
3137
государства
4058
OR
b'\x00\x00\x00\x00'
tf_idf 1.7711501702946344
b'\x01\x00\x00\x00'
tf_idf 1.7711501702946344
b'\x0b\x00\x00\x00'
tf_idf 1.1055454726302174
b'\x0c\x00\x00\x00'
tf_idf 1.1055454726302174
b'\r\x00\x00\x00'
tf_idf 1.438347821462426
b'\x0e\x00\x00\x00'
tf_idf 1.438347821462426
b'\x10\x00\x00\x00'
tf_idf 2.211090945260435
b'\x11\x00\x00\x00'
tf_idf 1.1055454726302174
b'\x12\x00\x00\x00'
tf_idf 1.1055454726302174
b'\x13\x00\x00\x00'
tf_idf 2.76955721679126
b'\x15\x00\x00\x00'
tf_idf 2.519265726773205
b'\x16\x00\x00\x00'
tf_idf 2.1605039587321606
b'\x19\x00\x00\x00'
tf_idf 2.405767839479198
b'\x1a\x00\x00\x00'
tf_idf 1.438347821462426
b'\x1f\x00\x00\x00'
tf_idf 1.438347821462426
b'.\x00\x00\x00'
tf_idf 1.1055454726302174
b'1\x00\x00\x00'
tf_idf 1.7711501702946344
b'2\x00\x00\x00'
tf_idf 1.1055454726302174
b'4\x00\x00\x00'
tf_idf 1.1055454726302174
b'5\x00\x00\x00'
tf_idf 1.10554547263021

tf_idf 1.438347821462426
b'\xf9\x0e\x00\x00'
tf_idf 2.1039525191268424
b'\xfa\x0e\x00\x00'
tf_idf 1.438347821462426
b'\xfc\x0e\x00\x00'
tf_idf 1.1055454726302174
b'\x01\x0f\x00\x00'
tf_idf 1.8782885964282263
b'\x02\x0f\x00\x00'
tf_idf 1.633024715681189
b'\x03\x0f\x00\x00'
tf_idf 1.7711501702946344
b'\x04\x0f\x00\x00'
tf_idf 1.438347821462426
b'\x05\x0f\x00\x00'
tf_idf 1.7711501702946344
b'\x06\x0f\x00\x00'
tf_idf 1.633024715681189
b'\n\x0f\x00\x00'
tf_idf 2.1605039587321606
b'\x0b\x0f\x00\x00'
tf_idf 1.1055454726302174
b'\x0c\x0f\x00\x00'
tf_idf 1.1055454726302174
b'\r\x0f\x00\x00'
tf_idf 1.438347821462426
b'\x11\x0f\x00\x00'
tf_idf 1.7711501702946344
b'\x12\x0f\x00\x00'
tf_idf 1.7711501702946344
b'\x13\x0f\x00\x00'
tf_idf 1.633024715681189
b'\x14\x0f\x00\x00'
tf_idf 1.438347821462426
b'\x15\x0f\x00\x00'
tf_idf 1.633024715681189
b'\x17\x0f\x00\x00'
tf_idf 1.633024715681189
b'\x19\x0f\x00\x00'
tf_idf 1.438347821462426
b'\x1a\x0f\x00\x00'
tf_idf 1.1055454726302174
b'\x1c\x0f\x00\x00'
tf_

b'\xc8)\x00\x00'
tf_idf 1.1055454726302174
b'\xcc)\x00\x00'
tf_idf 2.8520680756054135
b'\xda)\x00\x00'
tf_idf 1.1055454726302174
b'\xe9)\x00\x00'
tf_idf 1.1055454726302174
b'\xee)\x00\x00'
tf_idf 1.1055454726302174
b'\x03*\x00\x00'
tf_idf 1.7711501702946344
b'\x06*\x00\x00'
tf_idf 1.438347821462426
b'\x14*\x00\x00'
tf_idf 1.1055454726302174
b'\x18*\x00\x00'
tf_idf 1.438347821462426
b'\x1a*\x00\x00'
tf_idf 1.1055454726302174
b'\x1b*\x00\x00'
tf_idf 1.1055454726302174
b' *\x00\x00'
tf_idf 1.438347821462426
b'.*\x00\x00'
tf_idf 1.1055454726302174
b'1*\x00\x00'
tf_idf 1.7711501702946344
b'8*\x00\x00'
tf_idf 1.1055454726302174
b'A*\x00\x00'
tf_idf 1.1055454726302174
b'F*\x00\x00'
tf_idf 1.1055454726302174
b'n*\x00\x00'
tf_idf 1.1055454726302174
b'p*\x00\x00'
tf_idf 1.438347821462426
b'\x83*\x00\x00'
tf_idf 1.1055454726302174
b'\x8b*\x00\x00'
tf_idf 1.1055454726302174
b'\x9b*\x00\x00'
tf_idf 1.438347821462426
b'\x9e*\x00\x00'
tf_idf 1.1055454726302174
b'\x9f*\x00\x00'
tf_idf 1.77115017029463

tf_idf 1.1055454726302174
b'\x19I\x00\x00'
tf_idf 1.1055454726302174
b'\x1bI\x00\x00'
tf_idf 1.1055454726302174
b'\x1dI\x00\x00'
tf_idf 1.1055454726302174
b'%I\x00\x00'
tf_idf 1.438347821462426
b'(I\x00\x00'
tf_idf 1.1055454726302174
b'/I\x00\x00'
tf_idf 1.1055454726302174
b'0I\x00\x00'
tf_idf 1.1055454726302174
b'1I\x00\x00'
tf_idf 2.0398397846966496
b'5I\x00\x00'
tf_idf 1.1055454726302174
b'@I\x00\x00'
tf_idf 1.1055454726302174
b'KI\x00\x00'
tf_idf 1.1055454726302174
b'NI\x00\x00'
tf_idf 1.633024715681189
b'OI\x00\x00'
tf_idf 1.438347821462426
b'QI\x00\x00'
tf_idf 1.1055454726302174
b'XI\x00\x00'
tf_idf 1.1055454726302174
b'cI\x00\x00'
tf_idf 1.1055454726302174
b'uI\x00\x00'
tf_idf 1.1055454726302174
b'\x81I\x00\x00'
tf_idf 1.1055454726302174
b'\x84I\x00\x00'
tf_idf 1.1055454726302174
b'\x89I\x00\x00'
tf_idf 1.1055454726302174
b'\x8eI\x00\x00'
tf_idf 1.1055454726302174
b'\x8fI\x00\x00'
tf_idf 1.1055454726302174
b'\xb1I\x00\x00'
tf_idf 1.1055454726302174
b'\xb6I\x00\x00'
tf_idf 1.1055

b'7x\x00\x00'
tf_idf 1.8782885964282263
b'Cx\x00\x00'
tf_idf 1.7711501702946344
b'Wx\x00\x00'
tf_idf 1.633024715681189
b'ox\x00\x00'
tf_idf 1.1055454726302174
b'tx\x00\x00'
tf_idf 1.1055454726302174
b'ux\x00\x00'
tf_idf 1.438347821462426
b'\x85x\x00\x00'
tf_idf 1.1055454726302174
b'\x95x\x00\x00'
tf_idf 1.1055454726302174
b'\ry\x00\x00'
tf_idf 1.1055454726302174
b'\x10y\x00\x00'
tf_idf 1.438347821462426
b'\x12y\x00\x00'
tf_idf 1.1055454726302174
b'!y\x00\x00'
tf_idf 1.438347821462426
b'*y\x00\x00'
tf_idf 1.7711501702946344
b'.y\x00\x00'
tf_idf 1.633024715681189
b'3y\x00\x00'
tf_idf 1.438347821462426
b'8y\x00\x00'
tf_idf 1.1055454726302174
b'^y\x00\x00'
tf_idf 1.1055454726302174
b'\x80y\x00\x00'
tf_idf 1.1055454726302174
b'\x84y\x00\x00'
tf_idf 1.1055454726302174
b'\xacy\x00\x00'
tf_idf 1.1055454726302174
b'\xcey\x00\x00'
tf_idf 1.1055454726302174
b'\xecy\x00\x00'
tf_idf 1.1055454726302174
b'\xedy\x00\x00'
tf_idf 1.1055454726302174
b'\xeey\x00\x00'
tf_idf 1.1055454726302174
b'\xf1y\x00\

...

3.1009109324375634
Here
1.2928958892980056
2.432095770093061
Here
1.767034157423745
Here
1.2928958892980056
Here
0.9937479486306352
Here
1.5920438299653759
Here
2.1007268698762873
Here
1.2928958892980056
Here
0.9937479486306352
Here
0.9937479486306352
Here
0.9937479486306352
Here
0.9937479486306352
Here
1.8911917706327461
Here
1.1055454726302174
Here
0.9937479486306352
Here
1.1055454726302174
Here
1.1055454726302174
Here
0.9937479486306352
Here
1.1055454726302174
Here
1.2928958892980056
Here
1.633024715681189
Here
1.5920438299653759
Here
2.781838928538052
Here
1.1055454726302174
Here
1.633024715681189
Here
1.4678862167563744
Here
1.438347821462426
Here
1.2928958892980056
Here
1.2928958892980056
Here
1.2928958892980056
3.571838735883217
Here
1.9420244848821138
Here
1.1055454726302174
Here
0.9937479486306352
Here
0.9937479486306352
Here
1.1055454726302174
Here
1.1055454726302174
Here
1.1055454726302174
Here
1.1055454726302174
Here
1.438347821462426
Here
1.2928958892980056
Here
0.99374794

wiki16042 , 2.8720365450588616

wiki24116 , 2.8720365450588616

wiki24862 , 2.8720365450588616

wiki30775 , 2.8720365450588616

wiki10700 , 2.8520680756054135

wiki11745 , 2.839263788843385

wiki50 , 2.7938934292241173

wiki1160 , 2.7938934292241173

wiki1728 , 2.7938934292241173

wiki6596 , 2.7938934292241173

wiki6841 , 2.7938934292241173

wiki8406 , 2.7938934292241173

wiki11180 , 2.7938934292241173

wiki11229 , 2.7938934292241173

wiki11671 , 2.7938934292241173

wiki13154 , 2.7938934292241173

wiki16311 , 2.7938934292241173

wiki20806 , 2.7938934292241173

wiki21982 , 2.7938934292241173

wiki34375 , 2.7938934292241173

wiki34687 , 2.781838928538052

wiki332 , 2.7648981189252697

wiki828 , 2.7648981189252697

wiki1155 , 2.7648981189252697

wiki1171 , 2.7648981189252697

wiki1455 , 2.7648981189252697

wiki2401 , 2.7648981189252697

wiki2487 , 2.7648981189252697

wiki2488 , 2.7648981189252697

wiki2636 , 2.7648981189252697

wiki5344 , 2.7648981189252697

wiki6239 , 2.7648981189252697



wiki6030 , 1.438347821462426

wiki6073 , 1.438347821462426

wiki6104 , 1.438347821462426

wiki6187 , 1.438347821462426

wiki6192 , 1.438347821462426

wiki6194 , 1.438347821462426

wiki6203 , 1.438347821462426

wiki6208 , 1.438347821462426

wiki6211 , 1.438347821462426

wiki6236 , 1.438347821462426

wiki6242 , 1.438347821462426

wiki6310 , 1.438347821462426

wiki6312 , 1.438347821462426

wiki6329 , 1.438347821462426

wiki6349 , 1.438347821462426

wiki6350 , 1.438347821462426

wiki6351 , 1.438347821462426

wiki6410 , 1.438347821462426

wiki6578 , 1.438347821462426

wiki6611 , 1.438347821462426

wiki6617 , 1.438347821462426

wiki6620 , 1.438347821462426

wiki6621 , 1.438347821462426

wiki6622 , 1.438347821462426

wiki6623 , 1.438347821462426

wiki6625 , 1.438347821462426

wiki6626 , 1.438347821462426

wiki6627 , 1.438347821462426

wiki6628 , 1.438347821462426

wiki6629 , 1.438347821462426

wiki6630 , 1.438347821462426

wiki6631 , 1.438347821462426

wiki6632 , 1.438347821462426

wiki6633 

wiki31217 , 1.1055454726302174

wiki31230 , 1.1055454726302174

wiki31252 , 1.1055454726302174

wiki31285 , 1.1055454726302174

wiki31344 , 1.1055454726302174

wiki31351 , 1.1055454726302174

wiki31354 , 1.1055454726302174

wiki31373 , 1.1055454726302174

wiki31398 , 1.1055454726302174

wiki31406 , 1.1055454726302174

wiki31432 , 1.1055454726302174

wiki31436 , 1.1055454726302174

wiki31480 , 1.1055454726302174

wiki31481 , 1.1055454726302174

wiki31577 , 1.1055454726302174

wiki31624 , 1.1055454726302174

wiki31648 , 1.1055454726302174

wiki31678 , 1.1055454726302174

wiki31754 , 1.1055454726302174

wiki31766 , 1.1055454726302174

wiki31770 , 1.1055454726302174

wiki31774 , 1.1055454726302174

wiki31777 , 1.1055454726302174

wiki31883 , 1.1055454726302174

wiki31887 , 1.1055454726302174

wiki31889 , 1.1055454726302174

wiki31934 , 1.1055454726302174

wiki31964 , 1.1055454726302174

wiki31972 , 1.1055454726302174

wiki32017 , 1.1055454726302174

wiki32083 , 1.1055454726302174

wiki3209


wiki34119 , 0.9937479486306352

wiki34149 , 0.9937479486306352

wiki34157 , 0.9937479486306352

wiki34166 , 0.9937479486306352

wiki34176 , 0.9937479486306352

wiki34196 , 0.9937479486306352

wiki34211 , 0.9937479486306352

wiki34235 , 0.9937479486306352

wiki34250 , 0.9937479486306352

wiki34253 , 0.9937479486306352

wiki34268 , 0.9937479486306352

wiki34275 , 0.9937479486306352

wiki34301 , 0.9937479486306352

wiki34308 , 0.9937479486306352

wiki34322 , 0.9937479486306352

wiki34407 , 0.9937479486306352

wiki34409 , 0.9937479486306352

wiki34411 , 0.9937479486306352

wiki34412 , 0.9937479486306352

wiki34437 , 0.9937479486306352

wiki34475 , 0.9937479486306352

wiki34546 , 0.9937479486306352

wiki34563 , 0.9937479486306352

wiki34565 , 0.9937479486306352

wiki34566 , 0.9937479486306352

wiki34567 , 0.9937479486306352

wiki34627 , 0.9937479486306352

wiki34659 , 0.9937479486306352

wiki34758 , 0.9937479486306352

wiki34762 , 0.9937479486306352

wiki34798 , 0.9937479486306352

wiki348

In [15]:
a.search_bool('советского AND государства')

['советского', 'государства']
deque(['советского', 'государства', 'AND'])
советского
3137
государства
4058
AND
b'\x00\x00\x00\x00'
tf_idf 1.7711501702946344
b'\x01\x00\x00\x00'
tf_idf 1.7711501702946344
b'\x0b\x00\x00\x00'
tf_idf 1.1055454726302174
b'\x0c\x00\x00\x00'
tf_idf 1.1055454726302174
b'\r\x00\x00\x00'
tf_idf 1.438347821462426
b'\x0e\x00\x00\x00'
tf_idf 1.438347821462426
b'\x10\x00\x00\x00'
tf_idf 2.211090945260435
b'\x11\x00\x00\x00'
tf_idf 1.1055454726302174
b'\x12\x00\x00\x00'
tf_idf 1.1055454726302174
b'\x13\x00\x00\x00'
tf_idf 2.76955721679126
b'\x15\x00\x00\x00'
tf_idf 2.519265726773205
b'\x16\x00\x00\x00'
tf_idf 2.1605039587321606
b'\x19\x00\x00\x00'
tf_idf 2.405767839479198
b'\x1a\x00\x00\x00'
tf_idf 1.438347821462426
b'\x1f\x00\x00\x00'
tf_idf 1.438347821462426
b'.\x00\x00\x00'
tf_idf 1.1055454726302174
b'1\x00\x00\x00'
tf_idf 1.7711501702946344
b'2\x00\x00\x00'
tf_idf 1.1055454726302174
b'4\x00\x00\x00'
tf_idf 1.1055454726302174
b'5\x00\x00\x00'
tf_idf 1.105545472630

tf_idf 1.633024715681189
b'\xf4\x0e\x00\x00'
tf_idf 1.438347821462426
b'\xf5\x0e\x00\x00'
tf_idf 1.633024715681189
b'\xf6\x0e\x00\x00'
tf_idf 1.633024715681189
b'\xf7\x0e\x00\x00'
tf_idf 2.0398397846966496
b'\xf8\x0e\x00\x00'
tf_idf 1.438347821462426
b'\xf9\x0e\x00\x00'
tf_idf 2.1039525191268424
b'\xfa\x0e\x00\x00'
tf_idf 1.438347821462426
b'\xfc\x0e\x00\x00'
tf_idf 1.1055454726302174
b'\x01\x0f\x00\x00'
tf_idf 1.8782885964282263
b'\x02\x0f\x00\x00'
tf_idf 1.633024715681189
b'\x03\x0f\x00\x00'
tf_idf 1.7711501702946344
b'\x04\x0f\x00\x00'
tf_idf 1.438347821462426
b'\x05\x0f\x00\x00'
tf_idf 1.7711501702946344
b'\x06\x0f\x00\x00'
tf_idf 1.633024715681189
b'\n\x0f\x00\x00'
tf_idf 2.1605039587321606
b'\x0b\x0f\x00\x00'
tf_idf 1.1055454726302174
b'\x0c\x0f\x00\x00'
tf_idf 1.1055454726302174
b'\r\x0f\x00\x00'
tf_idf 1.438347821462426
b'\x11\x0f\x00\x00'
tf_idf 1.7711501702946344
b'\x12\x0f\x00\x00'
tf_idf 1.7711501702946344
b'\x13\x0f\x00\x00'
tf_idf 1.633024715681189
b'\x14\x0f\x00\x00'
tf_

tf_idf 1.438347821462426
b'3#\x00\x00'
tf_idf 1.438347821462426
b'4#\x00\x00'
tf_idf 1.438347821462426
b'5#\x00\x00'
tf_idf 1.438347821462426
b'6#\x00\x00'
tf_idf 1.438347821462426
b'T#\x00\x00'
tf_idf 1.7711501702946344
b'W#\x00\x00'
tf_idf 1.633024715681189
b'[#\x00\x00'
tf_idf 1.1055454726302174
b'a#\x00\x00'
tf_idf 1.438347821462426
b'n#\x00\x00'
tf_idf 1.438347821462426
b'o#\x00\x00'
tf_idf 1.1055454726302174
b'u#\x00\x00'
tf_idf 2.1605039587321606
b'\x98#\x00\x00'
tf_idf 2.2568524409371187
b'\x99#\x00\x00'
tf_idf 1.1055454726302174
b'\x9a#\x00\x00'
tf_idf 1.1055454726302174
b'\x9b#\x00\x00'
tf_idf 1.1055454726302174
b'\x9e#\x00\x00'
tf_idf 1.8782885964282263
b'\xa0#\x00\x00'
tf_idf 1.1055454726302174
b'\xa5#\x00\x00'
tf_idf 1.1055454726302174
b'\xa7#\x00\x00'
tf_idf 1.9658270645133973
b'\xa9#\x00\x00'
tf_idf 1.7711501702946344
b'\xb1#\x00\x00'
tf_idf 1.438347821462426
b'\xbb#\x00\x00'
tf_idf 1.633024715681189
b'\xbc#\x00\x00'
tf_idf 1.1055454726302174
b'\xc1#\x00\x00'
tf_idf 1.43

b'=?\x00\x00'
tf_idf 1.7711501702946344
b'F?\x00\x00'
tf_idf 1.1055454726302174
b'd?\x00\x00'
tf_idf 1.438347821462426
b'f?\x00\x00'
tf_idf 1.1055454726302174
b'j?\x00\x00'
tf_idf 1.438347821462426
b'z?\x00\x00'
tf_idf 1.1055454726302174
b'}?\x00\x00'
tf_idf 1.1055454726302174
b'\x84?\x00\x00'
tf_idf 1.1055454726302174
b'\x8a?\x00\x00'
tf_idf 1.1055454726302174
b'\x8d?\x00\x00'
tf_idf 1.1055454726302174
b'\x99?\x00\x00'
tf_idf 1.1055454726302174
b'\xa3?\x00\x00'
tf_idf 1.1055454726302174
b'\xac?\x00\x00'
tf_idf 1.1055454726302174
b'\xb7?\x00\x00'
tf_idf 1.1055454726302174
b'\xbd?\x00\x00'
tf_idf 1.1055454726302174
b'\xc9?\x00\x00'
tf_idf 1.1055454726302174
b'\xd2?\x00\x00'
tf_idf 1.1055454726302174
b'\xd5?\x00\x00'
tf_idf 1.1055454726302174
b'\xe0?\x00\x00'
tf_idf 1.9658270645133973
b'\xe5?\x00\x00'
tf_idf 1.1055454726302174
b'\xed?\x00\x00'
tf_idf 1.438347821462426
b'\x00@\x00\x00'
tf_idf 1.1055454726302174
b'\x0f@\x00\x00'
tf_idf 1.1055454726302174
b'\x11@\x00\x00'
tf_idf 2.465862706

tf_idf 1.633024715681189
b'\xa5n\x00\x00'
tf_idf 1.1055454726302174
b'\xa7n\x00\x00'
tf_idf 1.1055454726302174
b'\xb4n\x00\x00'
tf_idf 1.633024715681189
b'\xe0n\x00\x00'
tf_idf 1.1055454726302174
b'\xe6n\x00\x00'
tf_idf 1.1055454726302174
b'\xe9n\x00\x00'
tf_idf 1.1055454726302174
b'\xebn\x00\x00'
tf_idf 1.1055454726302174
b'\xf0n\x00\x00'
tf_idf 1.438347821462426
b'\xffn\x00\x00'
tf_idf 1.1055454726302174
b'\no\x00\x00'
tf_idf 1.438347821462426
b'\x0eo\x00\x00'
tf_idf 1.1055454726302174
b'*o\x00\x00'
tf_idf 1.438347821462426
b'<o\x00\x00'
tf_idf 1.1055454726302174
b'>o\x00\x00'
tf_idf 1.633024715681189
b'?o\x00\x00'
tf_idf 1.1055454726302174
b'Co\x00\x00'
tf_idf 1.633024715681189
b'Ro\x00\x00'
tf_idf 1.1055454726302174
b'\x88o\x00\x00'
tf_idf 1.1055454726302174
b'\x8co\x00\x00'
tf_idf 1.633024715681189
b'\xa0o\x00\x00'
tf_idf 1.1055454726302174
b'\xa3o\x00\x00'
tf_idf 1.1055454726302174
b'\xa8o\x00\x00'
tf_idf 1.1055454726302174
b'\xd1o\x00\x00'
tf_idf 1.1055454726302174
b'\xd2o\x00\x

...

tf_idf 1.4678862167563744
b'6H\x00\x00'
tf_idf 1.9874958972612704
b'8H\x00\x00'
tf_idf 1.6883479565939001
b'HH\x00\x00'
tf_idf 1.2928958892980056
b'QH\x00\x00'
tf_idf 1.2928958892980056
b'ZH\x00\x00'
tf_idf 0.9937479486306352
b'\x8bH\x00\x00'
tf_idf 0.9937479486306352
b'\x9aH\x00\x00'
tf_idf 0.9937479486306352
b'\x9fH\x00\x00'
tf_idf 0.9937479486306352
b'\xa8H\x00\x00'
tf_idf 1.4678862167563744
b'\xacH\x00\x00'
tf_idf 0.9937479486306352
b'\xb2H\x00\x00'
tf_idf 0.9937479486306352
b'\xb4H\x00\x00'
tf_idf 1.5920438299653759
b'\xbeH\x00\x00'
tf_idf 2.1624862247196397
b'\xc7H\x00\x00'
tf_idf 1.6883479565939001
b'\xe6H\x00\x00'
tf_idf 0.9937479486306352
b'\xeaH\x00\x00'
tf_idf 1.2928958892980056
b'\xeeH\x00\x00'
tf_idf 0.9937479486306352
b'\xf0H\x00\x00'
tf_idf 0.9937479486306352
b'\xf2H\x00\x00'
tf_idf 0.9937479486306352
b'\xf4H\x00\x00'
tf_idf 1.2928958892980056
b'\xf8H\x00\x00'
tf_idf 1.2928958892980056
b'\xfeH\x00\x00'
tf_idf 0.9937479486306352
b'\x05I\x00\x00'
tf_idf 1.4678862167563744


b'\x10h\x00\x00'
tf_idf 0.9937479486306352
b'\x13h\x00\x00'
tf_idf 0.9937479486306352
b'\x17h\x00\x00'
tf_idf 1.2928958892980056
b'\x1ah\x00\x00'
tf_idf 0.9937479486306352
b':h\x00\x00'
tf_idf 1.4678862167563744
b'Bh\x00\x00'
tf_idf 1.2928958892980056
b'Eh\x00\x00'
tf_idf 1.4678862167563744
b'Fh\x00\x00'
tf_idf 1.2928958892980056
b'_h\x00\x00'
tf_idf 0.9937479486306352
b'xh\x00\x00'
tf_idf 1.2928958892980056
b'\xaeh\x00\x00'
tf_idf 0.9937479486306352
b'\xb4h\x00\x00'
tf_idf 0.9937479486306352
b'\xb9h\x00\x00'
tf_idf 0.9937479486306352
b'\xc7h\x00\x00'
tf_idf 0.9937479486306352
b'\xdah\x00\x00'
tf_idf 0.9937479486306352
b'\xdfh\x00\x00'
tf_idf 0.9937479486306352
b'\xfbh\x00\x00'
tf_idf 0.9937479486306352
b'\xfdh\x00\x00'
tf_idf 0.9937479486306352
b'\xfeh\x00\x00'
tf_idf 1.767034157423745
b'\x14i\x00\x00'
tf_idf 0.9937479486306352
b'\x1bi\x00\x00'
tf_idf 0.9937479486306352
b'\x1ci\x00\x00'
tf_idf 1.4678862167563744
b'\x1di\x00\x00'
tf_idf 0.9937479486306352
b'\x1ei\x00\x00'
tf_idf 0.9937

tf_idf 1.4678862167563744
b'\xf9}\x00\x00'
tf_idf 0.9937479486306352
b'\xfd}\x00\x00'
tf_idf 0.9937479486306352
b'\x03~\x00\x00'
tf_idf 0.9937479486306352
b'\x08~\x00\x00'
tf_idf 0.9937479486306352
b'\x1f~\x00\x00'
tf_idf 2.0286297932255706
b')~\x00\x00'
tf_idf 1.2928958892980056
b'7~\x00\x00'
tf_idf 1.4678862167563744
b';~\x00\x00'
tf_idf 1.4678862167563744
b'[~\x00\x00'
tf_idf 0.9937479486306352
b'n~\x00\x00'
tf_idf 1.4678862167563744
b'o~\x00\x00'
tf_idf 0.9937479486306352
b'r~\x00\x00'
tf_idf 0.9937479486306352
b'\x87~\x00\x00'
tf_idf 0.9937479486306352
b'\x88~\x00\x00'
tf_idf 0.9937479486306352
b'\xb5~\x00\x00'
tf_idf 1.2928958892980056
b'\xba~\x00\x00'
tf_idf 1.4678862167563744
b'\xc7~\x00\x00'
tf_idf 0.9937479486306352
b'\xcc~\x00\x00'
tf_idf 1.6883479565939001
b'\xcf~\x00\x00'
tf_idf 0.9937479486306352
b'\xd5~\x00\x00'
tf_idf 0.9937479486306352
b'\xd8~\x00\x00'
tf_idf 1.5920438299653759
b'\xec~\x00\x00'
tf_idf 1.9874958972612704
b'\xf5~\x00\x00'
tf_idf 1.2928958892980056
b'\xf8

2.0992934212608527
3.785310013728778
2.0992934212608527
2.0992934212608527
2.939107864916791
2.0992934212608527
2.0992934212608527
2.9967372432629635
2.8725796300539623
3.126695778056326
2.9259206049791944
2.9595750131440326
2.7312437107604315
2.432095770093061
2.0992934212608527
3.524216486313935
2.0992934212608527
2.8720365450588616
2.7648981189252697
2.5734316893865916
3.06404605959264
2.432095770093061
2.9259206049791944
2.432095770093061
2.0992934212608527
2.9259206049791944
2.7938934292241173
2.8725796300539623
2.5734316893865916
2.7938934292241173
2.0992934212608527
2.432095770093061
3.453399848030166
2.8720365450588616
2.0992934212608527
2.0992934212608527
3.67952024701191
2.939107864916791
2.7312437107604315
3.3803723063445394
2.0992934212608527
3.4669776146879965
2.0992934212608527
2.9062340382188
2.8720365450588616
2.0992934212608527
3.205381978886171
2.0992934212608527
2.0992934212608527
2.7648981189252697
2.6975893025955933
3.1009109324375634
3.2250685456465646
2.099293421


wiki6859 , 2.432095770093061

wiki8642 , 2.432095770093061

wiki8842 , 2.432095770093061

wiki8843 , 2.432095770093061

wiki8892 , 2.432095770093061

wiki9372 , 2.432095770093061

wiki9536 , 2.432095770093061

wiki9788 , 2.432095770093061

wiki9789 , 2.432095770093061

wiki9889 , 2.432095770093061

wiki10758 , 2.432095770093061

wiki11511 , 2.432095770093061

wiki11565 , 2.432095770093061

wiki11649 , 2.432095770093061

wiki11801 , 2.432095770093061

wiki11841 , 2.432095770093061

wiki12581 , 2.432095770093061

wiki14178 , 2.432095770093061

wiki14902 , 2.432095770093061

wiki16151 , 2.432095770093061

wiki16593 , 2.432095770093061

wiki17203 , 2.432095770093061

wiki18163 , 2.432095770093061

wiki18591 , 2.432095770093061

wiki18873 , 2.432095770093061

wiki19283 , 2.432095770093061

wiki19759 , 2.432095770093061

wiki21075 , 2.432095770093061

wiki21334 , 2.432095770093061

wiki21944 , 2.432095770093061

wiki24328 , 2.432095770093061

wiki24760 , 2.432095770093061

wiki24794 , 2.432

In [17]:
a.search_bool('крымский OR мост')

['крымский', 'мост']
deque(['крымский', 'мост', 'OR'])
крымский
166
мост
1214
OR
b'\x19\x00\x00\x00'
tf_idf 2.3819519032879075
b'2\x00\x00\x00'
tf_idf 2.3819519032879075
b'4\x00\x00\x00'
tf_idf 2.3819519032879075
b'\x9d\x00\x00\x00'
tf_idf 2.3819519032879075
b'-\x01\x00\x00'
tf_idf 3.816029845525049
b'J\x01\x00\x00'
tf_idf 2.3819519032879075
b'c\x01\x00\x00'
tf_idf 2.3819519032879075
b'z\x01\x00\x00'
tf_idf 2.3819519032879075
b'[\x02\x00\x00'
tf_idf 2.3819519032879075
b'\xe4\x02\x00\x00'
tf_idf 2.3819519032879075
b'\x06\x03\x00\x00'
tf_idf 2.3819519032879075
b'#\x03\x00\x00'
tf_idf 3.098990874406478
b'8\x03\x00\x00'
tf_idf 2.3819519032879075
b'N\x03\x00\x00'
tf_idf 2.3819519032879075
b')\x06\x00\x00'
tf_idf 2.3819519032879075
b':\x06\x00\x00'
tf_idf 2.3819519032879075
b'I\x06\x00\x00'
tf_idf 2.3819519032879075
b'g\x06\x00\x00'
tf_idf 2.3819519032879075
b'\xc9\x06\x00\x00'
tf_idf 2.3819519032879075
b'^\x07\x00\x00'
tf_idf 2.3819519032879075
b'\xb1\x07\x00\x00'
tf_idf 3.518431784066522
b

tf_idf 1.5178413045887236
b't4\x00\x00'
tf_idf 1.5178413045887236
b'\x904\x00\x00'
tf_idf 1.9747570659276787
b'\x835\x00\x00'
tf_idf 1.9747570659276787
b'\x875\x00\x00'
tf_idf 1.9747570659276787
b'i7\x00\x00'
tf_idf 1.5178413045887236
b'x7\x00\x00'
tf_idf 2.4316728272666337
b'\x867\x00\x00'
tf_idf 1.9747570659276787
b'\x927\x00\x00'
tf_idf 1.5178413045887236
b'\x9d7\x00\x00'
tf_idf 1.5178413045887236
b'\x1c8\x00\x00'
tf_idf 1.9747570659276787
b'V8\x00\x00'
tf_idf 2.6989514136383796
b'g8\x00\x00'
tf_idf 1.5178413045887236
b'\xac8\x00\x00'
tf_idf 3.155867174977335
b'\xbd8\x00\x00'
tf_idf 1.5178413045887236
b'\xd88\x00\x00'
tf_idf 1.5178413045887236
b'\t9\x00\x00'
tf_idf 1.9747570659276787
b'\x0f9\x00\x00'
tf_idf 1.5178413045887236
b'59\x00\x00'
tf_idf 1.5178413045887236
b'F9\x00\x00'
tf_idf 4.182980686217195
b's9\x00\x00'
tf_idf 3.0985101364184384
b'y9\x00\x00'
tf_idf 3.035682609177447
b'\x899\x00\x00'
tf_idf 1.5178413045887236
b'\x919\x00\x00'
tf_idf 1.9747570659276787
b'\x939\x00\x00'


tf_idf 1.5178413045887236
b'E\x7f\x00\x00'
tf_idf 1.5178413045887236
b'\x8d\x7f\x00\x00'
tf_idf 3.6396923910882606
b'\x97\x7f\x00\x00'
tf_idf 2.578766847838492
b'\x9d\x7f\x00\x00'
tf_idf 2.2420356522994247
b'\xa6\x7f\x00\x00'
tf_idf 2.2420356522994247
b'\xa8\x7f\x00\x00'
tf_idf 1.5178413045887236
b'\xac\x7f\x00\x00'
tf_idf 2.2420356522994247
b'\xb6\x7f\x00\x00'
tf_idf 1.5178413045887236
b'\xbd\x7f\x00\x00'
tf_idf 1.5178413045887236
b'\xc2\x7f\x00\x00'
tf_idf 3.035682609177447
b'\xc5\x7f\x00\x00'
tf_idf 1.5178413045887236
b'\xc6\x7f\x00\x00'
tf_idf 1.9747570659276787
b'+\x80\x00\x00'
tf_idf 1.5178413045887236
b'>\x80\x00\x00'
tf_idf 2.578766847838492
b'C\x80\x00\x00'
tf_idf 3.0985101364184384
b'a\x80\x00\x00'
tf_idf 1.5178413045887236
b'y\x80\x00\x00'
tf_idf 1.9747570659276787
b'~\x80\x00\x00'
tf_idf 1.9747570659276787
b'\x97\x80\x00\x00'
tf_idf 1.5178413045887236
b'\xd6\x80\x00\x00'
tf_idf 1.5178413045887236
b'\xe2\x80\x00\x00'
tf_idf 1.5178413045887236
b'Y\x81\x00\x00'
tf_idf 1.517841

Here
1.9747570659276787
Here
1.5178413045887236
Here
1.5178413045887236
Here
1.5178413045887236
Here
1.5178413045887236
Here
2.3819519032879075
Here
1.5178413045887236
Here
1.5178413045887236
Here
1.5178413045887236
Here
5.183344716235859
4.813624730554541
Here
1.5178413045887236
Here
2.8005660161493364
Here
2.3819519032879075
Here
1.5178413045887236
Here
2.578766847838492
Here
2.3819519032879075
Here
1.5178413045887236
Here
1.5178413045887236
Here
1.5178413045887236
Here
1.9747570659276787
Here
3.0985101364184384
Here
1.5178413045887236
5.912776092753471
Here
2.4316728272666337
Here
1.5178413045887236
Here
2.3819519032879075
Here
1.5178413045887236
Here
1.5178413045887236
Here
1.5178413045887236
Here
2.3819519032879075
Here
2.3819519032879075
Here
1.5178413045887236
Here
2.4316728272666337
Here
2.6989514136383796
Here
1.5178413045887236
Here
1.5178413045887236
Here
1.5178413045887236
Here
2.4316728272666337
Here
1.5178413045887236
Here
2.2420356522994247
Here
1.5178413045887236
Here
1


wiki10074 , 3.0985101364184384

wiki14707 , 3.0985101364184384

wiki18729 , 3.0985101364184384

wiki22115 , 3.0985101364184384

wiki2035 , 3.035682609177447

wiki5622 , 3.035682609177447

wiki14713 , 3.035682609177447

wiki14902 , 3.035682609177447

wiki32706 , 3.035682609177447

wiki34343 , 3.035682609177447

wiki38559 , 3.035682609177447

wiki3058 , 2.9662300000101256

wiki6581 , 2.9662300000101256

wiki6976 , 2.9662300000101256

wiki7887 , 2.9662300000101256

wiki10014 , 2.9662300000101256

wiki10573 , 2.9662300000101256

wiki14824 , 2.9662300000101256

wiki17064 , 2.9662300000101256

wiki19363 , 2.9662300000101256

wiki22042 , 2.9662300000101256

wiki61 , 2.8885885886055886

wiki65 , 2.8885885886055886

wiki581 , 2.8885885886055886

wiki5920 , 2.8885885886055886

wiki10163 , 2.8885885886055886

wiki10627 , 2.8885885886055886

wiki11153 , 2.8885885886055886

wiki12797 , 2.8885885886055886

wiki17147 , 2.8885885886055886

wiki22043 , 2.8885885886055886

wiki24794 , 2.888588588605588

In [18]:
a.search_bool('крымский AND мост')

['крымский', 'мост']
deque(['крымский', 'мост', 'AND'])
крымский
166
мост
1214
AND
b'\x19\x00\x00\x00'
tf_idf 2.3819519032879075
b'2\x00\x00\x00'
tf_idf 2.3819519032879075
b'4\x00\x00\x00'
tf_idf 2.3819519032879075
b'\x9d\x00\x00\x00'
tf_idf 2.3819519032879075
b'-\x01\x00\x00'
tf_idf 3.816029845525049
b'J\x01\x00\x00'
tf_idf 2.3819519032879075
b'c\x01\x00\x00'
tf_idf 2.3819519032879075
b'z\x01\x00\x00'
tf_idf 2.3819519032879075
b'[\x02\x00\x00'
tf_idf 2.3819519032879075
b'\xe4\x02\x00\x00'
tf_idf 2.3819519032879075
b'\x06\x03\x00\x00'
tf_idf 2.3819519032879075
b'#\x03\x00\x00'
tf_idf 3.098990874406478
b'8\x03\x00\x00'
tf_idf 2.3819519032879075
b'N\x03\x00\x00'
tf_idf 2.3819519032879075
b')\x06\x00\x00'
tf_idf 2.3819519032879075
b':\x06\x00\x00'
tf_idf 2.3819519032879075
b'I\x06\x00\x00'
tf_idf 2.3819519032879075
b'g\x06\x00\x00'
tf_idf 2.3819519032879075
b'\xc9\x06\x00\x00'
tf_idf 2.3819519032879075
b'^\x07\x00\x00'
tf_idf 2.3819519032879075
b'\xb1\x07\x00\x00'
tf_idf 3.518431784066522

tf_idf 1.5178413045887236
b'h-\x00\x00'
tf_idf 1.5178413045887236
b'p-\x00\x00'
tf_idf 1.5178413045887236
b'\x89-\x00\x00'
tf_idf 1.9747570659276787
b'\x90-\x00\x00'
tf_idf 1.5178413045887236
b'\xa7-\x00\x00'
tf_idf 1.5178413045887236
b'\xde-\x00\x00'
tf_idf 1.5178413045887236
b'\xe1-\x00\x00'
tf_idf 3.155867174977335
b'\xe3-\x00\x00'
tf_idf 1.5178413045887236
b'\x01.\x00\x00'
tf_idf 1.9747570659276787
b'A.\x00\x00'
tf_idf 1.5178413045887236
b'F.\x00\x00'
tf_idf 1.5178413045887236
b'[.\x00\x00'
tf_idf 1.5178413045887236
b'q.\x00\x00'
tf_idf 2.2420356522994247
b't.\x00\x00'
tf_idf 1.5178413045887236
b'\xdf.\x00\x00'
tf_idf 1.5178413045887236
b'\x08/\x00\x00'
tf_idf 1.5178413045887236
b' /\x00\x00'
tf_idf 1.9747570659276787
b'-/\x00\x00'
tf_idf 1.5178413045887236
b'B/\x00\x00'
tf_idf 2.2420356522994247
b'E/\x00\x00'
tf_idf 1.9747570659276787
b'G/\x00\x00'
tf_idf 1.5178413045887236
b'\x81/\x00\x00'
tf_idf 1.5178413045887236
b'\x89/\x00\x00'
tf_idf 1.5178413045887236
b'\x94/\x00\x00'
tf_id

tf_idf 1.5178413045887236
b'\tu\x00\x00'
tf_idf 1.5178413045887236
b'\x1du\x00\x00'
tf_idf 2.4316728272666337
b'\x88u\x00\x00'
tf_idf 1.5178413045887236
b'\x8du\x00\x00'
tf_idf 1.5178413045887236
b'\x8eu\x00\x00'
tf_idf 1.5178413045887236
b'\x8fu\x00\x00'
tf_idf 1.5178413045887236
b'\xdcu\x00\x00'
tf_idf 1.5178413045887236
b'\x11v\x00\x00'
tf_idf 1.5178413045887236
b'\x16v\x00\x00'
tf_idf 1.5178413045887236
b' v\x00\x00'
tf_idf 1.5178413045887236
b'av\x00\x00'
tf_idf 1.5178413045887236
b'sv\x00\x00'
tf_idf 1.5178413045887236
b'{v\x00\x00'
tf_idf 1.9747570659276787
b'\x88v\x00\x00'
tf_idf 1.5178413045887236
b'\xc7v\x00\x00'
tf_idf 1.5178413045887236
b'{w\x00\x00'
tf_idf 1.9747570659276787
b'\xadw\x00\x00'
tf_idf 2.578766847838492
b'\xaew\x00\x00'
tf_idf 1.5178413045887236
b'\xfdw\x00\x00'
tf_idf 1.9747570659276787
b'<x\x00\x00'
tf_idf 1.5178413045887236
b'Bx\x00\x00'
tf_idf 3.257481777488292
b'\x8fx\x00\x00'
tf_idf 1.5178413045887236
b'\xb5x\x00\x00'
tf_idf 2.4316728272666337
b'\xe8x\x0

In [21]:
a.search_bool('Александр OR Пушкин')

['александр', 'пушкин']
deque(['александр', 'пушкин', 'OR'])
александр
4986
пушкин
463
OR
b'\x01\x00\x00\x00'
tf_idf 1.7209789616871338
b'\x0b\x00\x00\x00'
tf_idf 1.9678561188089254
b'\x0f\x00\x00\x00'
tf_idf 1.7672365824885115
b'\x10\x00\x00\x00'
tf_idf 1.9932027098627763
b'\x12\x00\x00\x00'
tf_idf 0.9043077171602077
b'\x13\x00\x00\x00'
tf_idf 1.448755213511492
b'\x14\x00\x00\x00'
tf_idf 1.3357721498243595
b'\x18\x00\x00\x00'
tf_idf 0.9043077171602077
b'\x19\x00\x00\x00'
tf_idf 1.1765314653358498
b'\x1a\x00\x00\x00'
tf_idf 0.9043077171602077
b'\x1c\x00\x00\x00'
tf_idf 1.5363916861447733
b'\x1f\x00\x00\x00'
tf_idf 0.9043077171602077
b' \x00\x00\x00'
tf_idf 0.9043077171602077
b'$\x00\x00\x00'
tf_idf 1.1765314653358498
b'&\x00\x00\x00'
tf_idf 1.3357721498243595
b"'\x00\x00\x00"
tf_idf 0.9043077171602077
b')\x00\x00\x00'
tf_idf 0.9043077171602077
b'*\x00\x00\x00'
tf_idf 1.1765314653358498
b'0\x00\x00\x00'
tf_idf 0.9043077171602077
b'1\x00\x00\x00'
tf_idf 1.5363916861447733
b'4\x00\x00\x00

b'\xad\t\x00\x00'
tf_idf 1.1765314653358498
b'\xae\t\x00\x00'
tf_idf 1.448755213511492
b'\xaf\t\x00\x00'
tf_idf 1.3357721498243595
b'\xb0\t\x00\x00'
tf_idf 1.3357721498243595
b'\xb1\t\x00\x00'
tf_idf 1.3357721498243595
b'\xb2\t\x00\x00'
tf_idf 1.3357721498243595
b'\xb3\t\x00\x00'
tf_idf 1.3357721498243595
b'\xb4\t\x00\x00'
tf_idf 1.7209789616871338
b'\xb5\t\x00\x00'
tf_idf 1.448755213511492
b'\xb6\t\x00\x00'
tf_idf 0.9043077171602077
b'\xb7\t\x00\x00'
tf_idf 1.7209789616871338
b'\xb8\t\x00\x00'
tf_idf 0.9043077171602077
b'\xb9\t\x00\x00'
tf_idf 1.1765314653358498
b'\xba\t\x00\x00'
tf_idf 1.448755213511492
b'\xbb\t\x00\x00'
tf_idf 1.5363916861447733
b'\xbc\t\x00\x00'
tf_idf 1.6079958980000018
b'\xbd\t\x00\x00'
tf_idf 0.9043077171602077
b'\xbe\t\x00\x00'
tf_idf 1.5363916861447733
b'\xbf\t\x00\x00'
tf_idf 1.448755213511492
b'\xc0\t\x00\x00'
tf_idf 1.7209789616871338
b'\xc1\t\x00\x00'
tf_idf 1.5363916861447733
b'\xc2\t\x00\x00'
tf_idf 0.9043077171602077
b'\xc3\t\x00\x00'
tf_idf 1.607995898

tf_idf 0.9043077171602077
b'_\x17\x00\x00'
tf_idf 1.448755213511492
b'c\x17\x00\x00'
tf_idf 1.448755213511492
b'|\x17\x00\x00'
tf_idf 1.1765314653358498
b'\x84\x17\x00\x00'
tf_idf 0.9043077171602077
b'\x89\x17\x00\x00'
tf_idf 1.1765314653358498
b'\x8b\x17\x00\x00'
tf_idf 1.3357721498243595
b'\x91\x17\x00\x00'
tf_idf 0.9043077171602077
b'\x9d\x17\x00\x00'
tf_idf 0.9043077171602077
b'\xa2\x17\x00\x00'
tf_idf 1.5363916861447733
b'\xa7\x17\x00\x00'
tf_idf 1.6079958980000018
b'\xc9\x17\x00\x00'
tf_idf 1.3357721498243595
b'\xd2\x17\x00\x00'
tf_idf 1.1765314653358498
b'\xd8\x17\x00\x00'
tf_idf 0.9043077171602077
b'\xdd\x17\x00\x00'
tf_idf 0.9043077171602077
b'\xec\x17\x00\x00'
tf_idf 1.5363916861447733
b'\xef\x17\x00\x00'
tf_idf 0.9043077171602077
b'\xf4\x17\x00\x00'
tf_idf 0.9043077171602077
b'\xf6\x17\x00\x00'
tf_idf 0.9043077171602077
b'\xfa\x17\x00\x00'
tf_idf 0.9043077171602077
b'\xfd\x17\x00\x00'
tf_idf 0.9043077171602077
b'\x03\x18\x00\x00'
tf_idf 0.9043077171602077
b'\x05\x18\x00\x00'

tf_idf 1.8802196461756442
b'P+\x00\x00'
tf_idf 1.448755213511492
b'Q+\x00\x00'
tf_idf 1.9678561188089254
b'R+\x00\x00'
tf_idf 0.9043077171602077
b'\x86+\x00\x00'
tf_idf 1.1765314653358498
b'\x91+\x00\x00'
tf_idf 1.7209789616871338
b'\x92+\x00\x00'
tf_idf 0.9043077171602077
b'\xa5+\x00\x00'
tf_idf 0.9043077171602077
b'\xaa+\x00\x00'
tf_idf 1.3357721498243595
b'\xac+\x00\x00'
tf_idf 1.8802196461756442
b'\xaf+\x00\x00'
tf_idf 0.9043077171602077
b'\xb4+\x00\x00'
tf_idf 0.9043077171602077
b'\xd3+\x00\x00'
tf_idf 0.9043077171602077
b'\xd4+\x00\x00'
tf_idf 0.9043077171602077
b'\xd9+\x00\x00'
tf_idf 1.3357721498243595
b'\xe8+\x00\x00'
tf_idf 1.3357721498243595
b'\xef+\x00\x00'
tf_idf 1.3357721498243595
b'\xf2+\x00\x00'
tf_idf 0.9043077171602077
b'\xfa+\x00\x00'
tf_idf 0.9043077171602077
b'\x00,\x00\x00'
tf_idf 0.9043077171602077
b'\x01,\x00\x00'
tf_idf 0.9043077171602077
b'\x06,\x00\x00'
tf_idf 1.1765314653358498
b'\x07,\x00\x00'
tf_idf 0.9043077171602077
b'\x0f,\x00\x00'
tf_idf 1.448755213511

tf_idf 1.3357721498243595
b'l?\x00\x00'
tf_idf 0.9043077171602077
b'v?\x00\x00'
tf_idf 1.6079958980000018
b'z?\x00\x00'
tf_idf 1.7672365824885115
b'{?\x00\x00'
tf_idf 1.1765314653358498
b'}?\x00\x00'
tf_idf 1.6079958980000018
b'\x7f?\x00\x00'
tf_idf 0.9043077171602077
b'\x86?\x00\x00'
tf_idf 1.1765314653358498
b'\x88?\x00\x00'
tf_idf 0.9043077171602077
b'\x8b?\x00\x00'
tf_idf 1.7672365824885115
b'\x8d?\x00\x00'
tf_idf 0.9043077171602077
b'\x8e?\x00\x00'
tf_idf 1.9407601446777085
b'\x99?\x00\x00'
tf_idf 1.1765314653358498
b'\x9c?\x00\x00'
tf_idf 0.9043077171602077
b'\xac?\x00\x00'
tf_idf 1.3357721498243595
b'\xb0?\x00\x00'
tf_idf 0.9043077171602077
b'\xb1?\x00\x00'
tf_idf 0.9043077171602077
b'\xbd?\x00\x00'
tf_idf 1.8802196461756442
b'\xc0?\x00\x00'
tf_idf 1.3357721498243595
b'\xc7?\x00\x00'
tf_idf 0.9043077171602077
b'\xd2?\x00\x00'
tf_idf 1.1765314653358498
b'\xd5?\x00\x00'
tf_idf 0.9043077171602077
b'\xe5?\x00\x00'
tf_idf 0.9043077171602077
b'\xe8?\x00\x00'
tf_idf 1.3357721498243595


...

0.9043077171602077
Here
0.9043077171602077
Here
1.1765314653358498
Here
1.3357721498243595
Here
1.1765314653358498
Here
1.1765314653358498
Here
0.9043077171602077
Here
0.9043077171602077
Here
1.3357721498243595
Here
1.3357721498243595
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
4.5511107439549265
Here
1.1765314653358498
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
1.5363916861447733
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
3.113010465645859
Here
0.9043077171602077
Here
1.3357721498243595
Here
1.9364790003100092
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9

0.9043077171602077
Here
0.9043077171602077
Here
1.448755213511492
Here
1.1765314653358498
Here
0.9043077171602077
Here
1.1765314653358498
Here
1.448755213511492
Here
1.1765314653358498
Here
0.9043077171602077
Here
0.9043077171602077
Here
1.1765314653358498
Here
0.9043077171602077
Here
1.9364790003100092
Here
1.1765314653358498
Here
1.1765314653358498
3.3852342138215015
Here
0.9043077171602077
Here
1.5363916861447733
3.42372498253693
Here
0.9043077171602077
Here
1.1765314653358498
3.42372498253693
Here
0.9043077171602077
Here
1.3357721498243595
Here
1.3357721498243595
Here
0.9043077171602077
Here
0.9043077171602077
Here
1.9678561188089254
Here
0.9043077171602077
Here
0.9043077171602077
Here
2.1000008291662184
Here
1.1765314653358498
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
0.9043077171602077
Here
1.448755213511492
Here
0.9043077171602077
Here
1.1765314653358498
Here
0.9043077171602077
4.196186440500558
Here
1.5363916861447733
Here
1.448755213511492
He


wiki29253 , 1.9407601446777085

wiki31257 , 1.9407601446777085

wiki32009 , 1.9407601446777085

wiki32595 , 1.9407601446777085

wiki32821 , 1.9407601446777085

wiki34979 , 1.9407601446777085

wiki35046 , 1.9407601446777085

wiki193 , 1.9364790003100092

wiki370 , 1.9364790003100092

wiki591 , 1.9364790003100092

wiki826 , 1.9364790003100092

wiki1140 , 1.9364790003100092

wiki1385 , 1.9364790003100092

wiki1532 , 1.9364790003100092

wiki1758 , 1.9364790003100092

wiki1847 , 1.9364790003100092

wiki1934 , 1.9364790003100092

wiki4017 , 1.9364790003100092

wiki4051 , 1.9364790003100092

wiki4052 , 1.9364790003100092

wiki4171 , 1.9364790003100092

wiki4385 , 1.9364790003100092

wiki4660 , 1.9364790003100092

wiki4905 , 1.9364790003100092

wiki4926 , 1.9364790003100092

wiki5077 , 1.9364790003100092

wiki5089 , 1.9364790003100092

wiki5294 , 1.9364790003100092

wiki5432 , 1.9364790003100092

wiki5529 , 1.9364790003100092

wiki6215 , 1.9364790003100092

wiki6541 , 1.9364790003100092

wiki

wiki25344 , 1.1765314653358498

wiki25348 , 1.1765314653358498

wiki25469 , 1.1765314653358498

wiki25754 , 1.1765314653358498

wiki25821 , 1.1765314653358498

wiki25825 , 1.1765314653358498

wiki25826 , 1.1765314653358498

wiki25884 , 1.1765314653358498

wiki25942 , 1.1765314653358498

wiki25976 , 1.1765314653358498

wiki26013 , 1.1765314653358498

wiki26025 , 1.1765314653358498

wiki26303 , 1.1765314653358498

wiki26319 , 1.1765314653358498

wiki26729 , 1.1765314653358498

wiki26878 , 1.1765314653358498

wiki26931 , 1.1765314653358498

wiki26957 , 1.1765314653358498

wiki27001 , 1.1765314653358498

wiki27084 , 1.1765314653358498

wiki27091 , 1.1765314653358498

wiki27102 , 1.1765314653358498

wiki27157 , 1.1765314653358498

wiki27199 , 1.1765314653358498

wiki27202 , 1.1765314653358498

wiki27289 , 1.1765314653358498

wiki27345 , 1.1765314653358498

wiki27510 , 1.1765314653358498

wiki27512 , 1.1765314653358498

wiki27561 , 1.1765314653358498

wiki27567 , 1.1765314653358498

wiki2759


wiki27206 , 0.9043077171602077

wiki27210 , 0.9043077171602077

wiki27288 , 0.9043077171602077

wiki27291 , 0.9043077171602077

wiki27339 , 0.9043077171602077

wiki27346 , 0.9043077171602077

wiki27368 , 0.9043077171602077

wiki27420 , 0.9043077171602077

wiki27432 , 0.9043077171602077

wiki27438 , 0.9043077171602077

wiki27442 , 0.9043077171602077

wiki27460 , 0.9043077171602077

wiki27504 , 0.9043077171602077

wiki27506 , 0.9043077171602077

wiki27523 , 0.9043077171602077

wiki27563 , 0.9043077171602077

wiki27593 , 0.9043077171602077

wiki27599 , 0.9043077171602077

wiki27604 , 0.9043077171602077

wiki27613 , 0.9043077171602077

wiki27615 , 0.9043077171602077

wiki27616 , 0.9043077171602077

wiki27625 , 0.9043077171602077

wiki27628 , 0.9043077171602077

wiki27671 , 0.9043077171602077

wiki27680 , 0.9043077171602077

wiki27716 , 0.9043077171602077

wiki27724 , 0.9043077171602077

wiki27729 , 0.9043077171602077

wiki27745 , 0.9043077171602077

wiki27766 , 0.9043077171602077

wiki277